In [884]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import sys

In [885]:
ruta = "Mexico/"
archivo = 'LIVEPOOLC1.xlsx'
ruta_archivo = ruta + archivo
anio_actual = datetime.now().year
anio_empieza_analisis = anio_actual - 10

In [886]:
# Reemplaza esto con la ruta a tu archivo
ruta_archivo = ruta_archivo
df_err = pd.read_excel(ruta_archivo, header=19)
# Filas a eliminar (ajustar índices para que sean 0-based)
filas_a_eliminar = [1, 16, 53, 86, 150, 191, 212]
# Eliminar las filas
df_err = df_err.drop(filas_a_eliminar)
df_err.columns = df_err.iloc[0]
if '\t' in df_err.columns:
    df = df_err.drop(columns=['\t'])
df_err = df_err[1:].reset_index(drop=True)

In [887]:
def calcular_crecimiento_ipc(archivo_csv, anio_cierre, periodo_anos=5):
    df = pd.read_csv(f"{archivo_csv}.csv")
    df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d.%m.%Y')
    df['Cierre'] = df['Cierre'].str.replace(',', '').astype(float)
    df = df.sort_values('Fecha')
    
    # Filtrar datos del año de cierre
    df_ano = df[df['Fecha'].dt.year == anio_cierre]
    if df_ano.empty:
        raise ValueError(f"No hay datos para el año {anio_cierre}")
    fecha_final = df_ano['Fecha'].max()
    fecha_inicio = fecha_final - pd.DateOffset(years=periodo_anos)
    
    df_filtrado = df[(df['Fecha'] >= fecha_inicio) & (df['Fecha'] <= fecha_final)]
    if df_filtrado.shape[0] < 2:
        raise ValueError("Datos insuficientes para el período solicitado")
    
    precio_inicial = df_filtrado.iloc[0]['Cierre']
    precio_final = df_filtrado.iloc[-1]['Cierre']
    
    tasa = ((precio_final / precio_inicial) ** (1 / periodo_anos)) - 1
    return fecha_inicio.date(), fecha_final.date(), tasa 

# Ejemplo de uso:
archivo = "Datos históricos S&P_BMV IRT"

años_objetivo = [2023, 2022, 2021, 2020, 2019,2018, 2017, 2016, 2015, 2014,2013]

crecimiento_ipc_porcentaje_años = []

for año in años_objetivo:
    fi, ff, tasa = calcular_crecimiento_ipc(archivo, año)
    crecimiento_ipc_porcentaje_años.append({
        "año_cierre": año,
        "fecha_inicio": fi,
        "fecha_final": ff,
        "CAGR_%": round(tasa, 4)
    })

crecimiento_ipc_porcentaje_años


[{'año_cierre': 2023,
  'fecha_inicio': datetime.date(2018, 12, 17),
  'fecha_final': datetime.date(2023, 12, 17),
  'CAGR_%': np.float64(0.0998)},
 {'año_cierre': 2022,
  'fecha_inicio': datetime.date(2017, 12, 25),
  'fecha_final': datetime.date(2022, 12, 25),
  'CAGR_%': np.float64(0.0225)},
 {'año_cierre': 2021,
  'fecha_inicio': datetime.date(2016, 12, 26),
  'fecha_final': datetime.date(2021, 12, 26),
  'CAGR_%': np.float64(0.0558)},
 {'año_cierre': 2020,
  'fecha_inicio': datetime.date(2015, 12, 27),
  'fecha_final': datetime.date(2020, 12, 27),
  'CAGR_%': np.float64(0.0286)},
 {'año_cierre': 2019,
  'fecha_inicio': datetime.date(2014, 12, 29),
  'fecha_final': datetime.date(2019, 12, 29),
  'CAGR_%': np.float64(0.0335)},
 {'año_cierre': 2018,
  'fecha_inicio': datetime.date(2013, 12, 30),
  'fecha_final': datetime.date(2018, 12, 30),
  'CAGR_%': np.float64(0.0185)},
 {'año_cierre': 2017,
  'fecha_inicio': datetime.date(2012, 12, 31),
  'fecha_final': datetime.date(2017, 12, 31

In [888]:
archivo_market_cetes = "cetes_10_anios"
data_cetes = pd.read_excel(archivo_market_cetes + '.xlsx' , header=17)

data_cetes['Fecha'] = pd.to_datetime(data_cetes['Fecha'], format='%d/%m/%Y')
data_cetes = data_cetes.rename(columns={data_cetes.columns[1]: 'Rendimiento'})

años = [2023, 2022, 2021, 2020, 2019 , 2018, 2017, 2016, 2015, 2014, 2013]
data_cetes = data_cetes[data_cetes['Fecha'].dt.year.isin(años)]

promedio_cetes = (
    data_cetes
    .groupby(data_cetes['Fecha'].dt.year)['Rendimiento']
    .mean()
    .reset_index()
    .rename(columns={'Fecha': 'Año', 'Rendimiento': 'Promedio_Anual'})
)
promedio_cetes['Promedio_Anual'] = promedio_cetes['Promedio_Anual'].round(4)
print(promedio_cetes)

     Año  Promedio_Anual
0   2013          3.9327
1   2014          3.2781
2   2015          3.4980
3   2016          4.6900
4   2017          7.0847
5   2018          8.0373
6   2019          7.8175
7   2020          5.2056
8   2021          5.2381
9   2022          9.2586
10  2023         11.4710


c:\Users\mar_c\OneDrive\Documentos\GuruFocus\guru_focus\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [889]:
archivo_inflacion_mexico = "INFLACION_mexico"
data_inflacion_mexico = pd.read_excel(archivo_inflacion_mexico + '.xlsx')
# Años de interés
años = [2023, 2022, 2021, 2020, 2019 , 2018, 2017, 2016, 2015, 2014, 2013]

# Filtrado y extracción
resultados_inf_mexico = []
for año in años:
    fila = data_inflacion_mexico[data_inflacion_mexico['Año'] == año]
    if not fila.empty:
        tasa = fila.iloc[0]['Tasa de Inflación']/100
        resultados_inf_mexico.append({"Año": año, "Tasa de Inflación": round(tasa, 4)})
    else:
        resultados_inf_mexico.append({"Año": año, "Tasa de Inflacion": None})

# Convertir en DataFrame (o lo que prefieras)
df_resultados_inf_mexico = pd.DataFrame(resultados_inf_mexico)
print(df_resultados_inf_mexico)


     Año  Tasa de Inflación
0   2023             0.0466
1   2022             0.0782
2   2021             0.0736
3   2020             0.0315
4   2019             0.0283
5   2018             0.0483
6   2017             0.0677
7   2016             0.0336
8   2015             0.0213
9   2014             0.0408
10  2013             0.0397


In [890]:
archivo_stock = "Datos históricos LIVEPOLC1_20anios"
archivo_market = "Datos históricos S&P_BMV IRT"

In [891]:
data_stock = pd.read_csv(archivo_stock + '.csv')
data_stock['Fecha'] = pd.to_datetime(data_stock['Fecha'], format='%d.%m.%Y')
data_stock['Cierre'] = data_stock['Cierre'].astype(str).str.replace(',', '').astype(float)
data_stock = data_stock.dropna(subset=['Cierre'])
# Agregar columna de año
data_stock["Año"] = data_stock["Fecha"].dt.year

# Filtrar solo entre 2013 y 2023
data_filtrada = data_stock[(data_stock["Año"] >= 2013) & (data_stock["Año"] <= 2023)]

# Calcular promedio y mediana por año
resumen_cierre = data_filtrada.groupby("Año")["Cierre"].agg(
    Promedio_Cierre="mean",
    Mediana_Cierre="median"
).reset_index()
resumen_cierre

,Año,Promedio_Cierre,Mediana_Cierre
0,2013,146.881538,147.065
1,2014,146.560962,146.975
2,2015,190.523269,185.550
3,2016,191.624615,197.615
4,2017,142.654340,143.640
5,2018,130.687308,131.885
6,2019,109.000577,107.695
7,2020,67.014808,60.770
8,2021,82.280577,86.985
9,2022,98.880000,98.625


In [892]:

# Supuesto: ya tienes 'promedio_cetes' con columnas Año y Promedio_Anual (en %, tipo float)
promedio_cetes['Promedio_Anual'] = promedio_cetes['Promedio_Anual'] / 100

# Carga de datos
df_stock = pd.read_csv(f"{archivo_stock}.csv")
df_market = pd.read_csv(f"{archivo_market}.csv")

# Limpieza y rendimientos semanales
for df in (df_stock, df_market):
    df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d.%m.%Y')
    # Solo aplicar str.replace si el tipo es object (string)
    if df['Cierre'].dtype == object:
        df['Cierre'] = df['Cierre'].str.replace(',', '').astype(float)
    else:
        df['Cierre'] = df['Cierre'].astype(float)
    df.sort_values('Fecha', inplace=True)
    df['Rend'] = df['Cierre'].pct_change()
    df.dropna(subset=['Rend'], inplace=True)

df_stock.rename(columns={'Rend': 'Rend_Stock'}, inplace=True)
df_market.rename(columns={'Rend': 'Rend_Market'}, inplace=True)

# Función para calcular beta anual
def calcular_beta_anual(anio):
    rf_row = promedio_cetes.loc[promedio_cetes['Año'] == anio, 'Promedio_Anual']
    if rf_row.empty:
        return None
    rf_rate = rf_row.iloc[0] / 52

    s = df_stock[df_stock['Fecha'].dt.year == anio]
    m = df_market[df_market['Fecha'].dt.year == anio]
    if s.empty or m.empty:
        return None

    merged = pd.merge(
        s[['Fecha', 'Rend_Stock']],
        m[['Fecha', 'Rend_Market']],
        on='Fecha'
    ).dropna()

    merged['Excess_Stock'] = merged['Rend_Stock'] - rf_rate
    merged['Excess_Market'] = merged['Rend_Market'] - rf_rate

    cov = np.cov(merged['Excess_Stock'], merged['Excess_Market'])[0, 1]
    var_m = np.var(merged['Excess_Market'])
    return cov/var_m if var_m != 0 else None

# Cálculo de betas para los años en promedio_cetes
resultados_beta = []
for ano in promedio_cetes['Año']:
    beta = calcular_beta_anual(int(ano))
    resultados_beta.append({'Año': int(ano), 'Beta': round(beta, 4) if beta is not None else None})

df_beta = pd.DataFrame(resultados_beta)
print(df_beta)


     Año    Beta
0   2013  0.8611
1   2014  0.7742
2   2015  0.7197
3   2016  0.8511
4   2017  1.6133
5   2018  1.0291
6   2019  1.0586
7   2020  1.3802
8   2021  0.5047
9   2022  0.3051
10  2023  0.7428


In [893]:
# Obtengo los DF para datos anulaes y trimestrales
indice_ttm = df_err.columns.get_loc('TTM/current')
# Crear una copia del DataFrame para datos anuales
df_anual = df_err.iloc[:, :indice_ttm + 1]
# Crear una copia del DataFrame para datos trimestrales
df_trimestral = df_err.iloc[:, indice_ttm + 1:]
df_trimestral.insert(0, 'Fiscal Period', df_err['Fiscal Period'])


In [894]:
columna_anterior = df_anual.columns[indice_ttm - 1]
anio_anterior = int(columna_anterior[-4:])
# Identificar las columnas que contienen un año en su nombre
pattern = re.compile(r'\d{4}')
columnas_con_anio = [col for col in df_anual.columns if pattern.search(col)]

# Filtrar las columnas que están dentro del rango de los últimos 10 años desde anio_anterior
anio_limite = anio_anterior - 20
columnas_filtradas = [col for col in columnas_con_anio if int(
    pattern.search(col).group()) >= anio_limite]

# Asegurarse de incluir 'Fiscal Period' y 'TTM/current' si están presentes
columnas_especiales = ['Fiscal Period', 'TTM/current']
columnas_filtradas = [
    col for col in df_anual.columns if col in columnas_especiales] + columnas_filtradas

# Crear un nuevo DataFrame con las columnas filtradas
df_anual_filtrado = df_anual[columnas_filtradas]

# Mover la columna 'TTM/current' al final
if 'TTM/current' in df_anual_filtrado.columns:
    columnas_ordenadas = [
        col for col in df_anual_filtrado.columns if col != 'TTM/current'] + ['TTM/current']
    df_anual_filtrado = df_anual_filtrado[columnas_ordenadas]

df_anual_filtrado = df_anual_filtrado.fillna(0)

# df_anual_filtrado = df_anual_filtrado.loc[:, ~df_anual_filtrado.columns.str.contains("2020")]

In [895]:
pattern = re.compile(r'\d{4}')
columnas_con_anio_trimestral = [
    col for col in df_trimestral.columns if pattern.search(col)]

# Obtener el año de la última columna con año en su nombre
ultima_columna = columnas_con_anio_trimestral[-1]
anio_ultima_columna = int(pattern.search(ultima_columna).group())

# Filtrar las columnas que están dentro del rango de los últimos 10 años desde anio_ultima_columna
anio_limite_trimestral = anio_ultima_columna - 20
columnas_filtradas_trimestral = [col for col in columnas_con_anio_trimestral if int(
    pattern.search(col).group()) >= anio_limite_trimestral]

# Asegurarse de incluir 'Fiscal Period' si está presente
columnas_especiales_trimestral = ['Fiscal Period']

# Crear un nuevo DataFrame con las columnas filtradas
df_trimestral_filtrado = df_trimestral[columnas_especiales_trimestral +
                                       columnas_filtradas_trimestral]

df_trimestral_filtrado = df_trimestral_filtrado.fillna(0)

In [896]:
df_anual_filtrado = df_anual_filtrado.drop(columns=["TTM/current"])

In [897]:
def limpiar_df_financiero(df):
    df = df.copy()

    # Reemplazar '-' por NaN en todo el DF
    df.replace('-', np.nan, inplace=True)

    # Todas las columnas excepto 'Fiscal Period' a numéricas
    cols_datos = df.columns.drop('Fiscal Period')

    df[cols_datos] = df[cols_datos].apply(
        pd.to_numeric, errors='coerce'
    )

    return df

df_anual_filtrado = limpiar_df_financiero(df_anual_filtrado)
df_trimestral_filtrado = limpiar_df_financiero(df_trimestral_filtrado)

C:\Users\mar_c\AppData\Local\Temp\ipykernel_15112\1578466456.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('-', np.nan, inplace=True)
C:\Users\mar_c\AppData\Local\Temp\ipykernel_15112\1578466456.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('-', np.nan, inplace=True)


In [898]:
df_anual_filtrado = df_anual_filtrado.dropna(axis=1, how='all')
df_trimestral_filtrado = df_trimestral_filtrado.dropna(axis=1, how='all')

In [899]:
# Crear DataFrame vacío con las columnas anuales
razones_financieras = pd.DataFrame(
    columns=df_anual_filtrado.columns.difference(['Fiscal Period']))


def agregar_razon_financiera(df_anual_filtrado, fila_numerador, nombre_nueva_fila, *filas_denominadoras, como_porcentaje=True):

    global razones_financieras

    # Obtener la fila numerador
    fila_num = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_numerador].iloc[0, 1:]

    # Obtener y sumar las filas denominadoras
    fila_den = sum(df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila].iloc[0, 1:].values for fila in filas_denominadoras)

    # Calcular la razón financiera, manejando la división por cero
    razon_financiera = []
    for num, den in zip(fila_num.values, fila_den):
        if den != 0:
            razon = num / den
            if como_porcentaje:
                razon *= 100
            razon_financiera.append(f"{razon:.2f}")
        else:
            razon_financiera.append("no deuda")

    # Agregar el resultado al DataFrame
    razones_financieras.loc[nombre_nueva_fila] = razon_financiera

def agregar_razon_financiera_numerador(df_anual_filtrado, nombre_nueva_fila, fila_denominador, fila_numerador_1, fila_numerador_2, como_porcentaje=True):

    global razones_financieras

    # Obtener las filas numeradoras
    fila_num_1 = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_numerador_1].iloc[0, 1:]
    fila_num_2 = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_numerador_2].iloc[0, 1:]

    # Obtener la fila denominador
    fila_den = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_denominador].iloc[0, 1:]

    # Restar las filas numeradoras
    resta_numeradores = fila_num_1.values - fila_num_2.values

    # Calcular la razón financiera, manejando la división por cero
    razon_financiera = []
    for num, den in zip(resta_numeradores, fila_den.values):
        if den != 0:
            razon = num / den
            if como_porcentaje:
                razon *= 100
            razon_financiera.append(f"{razon:.2f}")
        else:
            razon_financiera.append("no deuda")

    # Agregar el resultado al DataFrame razones_financieras
    razones_financieras.loc[nombre_nueva_fila] = razon_financiera

def agregar_fila(df_anual_filtrado, fila_nombre, nombre_nueva_fila):

    global razones_financieras

    # Obtener la fila relevante
    fila = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == fila_nombre].iloc[0, 1:]

    # Agregar el resultado al DataFrame
    razones_financieras.loc[nombre_nueva_fila] = fila.values

def agregar_crecimiento_anual(fila_nombre, nombre_nueva_fila):

    global razones_financieras

    # Obtener la fila relevante
    fila = razones_financieras.loc[fila_nombre].astype(float)

    # Calcular el crecimiento año a año (YoY)
    crecimiento_anual = fila.pct_change() * 100

    # Reemplazar NaN por 0
    crecimiento_anual = crecimiento_anual.fillna(0)

    # Formatear el resultado a dos decimales
    crecimiento_anual_formateado = [f"{x:.2f}" for x in crecimiento_anual]

    # Agregar el resultado al DataFrame
    razones_financieras.loc[nombre_nueva_fila] = crecimiento_anual_formateado

def dividir_filas_razones_financieras(fila_numerador, fila_denominador, nombre_nueva_fila, como_porcentaje=True):
    global razones_financieras

    # Obtener las filas relevantes para el cálculo
    fila_num = razones_financieras.loc[fila_numerador].astype(float)
    fila_den = razones_financieras.loc[fila_denominador].astype(float)

    # Calcular la razón financiera
    razon_financiera = fila_num.values / fila_den.values

    # Convertir a porcentaje si es necesario
    if como_porcentaje:
        razon_financiera *= 100

    # Reemplazar NaN por 0
    razon_financiera = pd.Series(razon_financiera).fillna(0).values

    # Formatear el resultado a dos decimales
    razon_financiera_formateada = [f"{x:.2f}" for x in razon_financiera]

    # Agregar el resultado al DataFrame
    razones_financieras.loc[nombre_nueva_fila] = razon_financiera_formateada



In [900]:
# Llamadas a la función con el nuevo orden de parámetros
agregar_razon_financiera(df_anual_filtrado, 'Depreciation, Depletion and Amortization', 'Amortizacion(%)', 'Gross Profit', como_porcentaje=True)
agregar_razon_financiera(df_anual_filtrado, 'Net Income', 'Beneficio Neto(%)', 'Revenue', como_porcentaje=True)
agregar_fila(df_anual_filtrado, 'Net Income', 'Beneficio Neto')
agregar_razon_financiera(df_anual_filtrado, 'Gross Profit', 'Margen Bruto(%)', 'Revenue', como_porcentaje=True)
agregar_razon_financiera(df_anual_filtrado, 'Operating Income', 'Margen Operativo(%)', 'Revenue', como_porcentaje=True)
agregar_razon_financiera(df_anual_filtrado, 'Net Income', 'EPS', 'Shares Outstanding (Diluted Average)', como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado, 'Revenue', 'Revenue Per Share', 'Shares Outstanding (Diluted Average)', como_porcentaje=False)
agregar_fila(df_anual_filtrado, 'Shares Outstanding (Diluted Average)', '# de Acciones')
agregar_fila(df_anual_filtrado, 'Revenue', 'Ventas')
agregar_crecimiento_anual('Beneficio Neto', 'Crecimiento Beneficio Neto')
agregar_crecimiento_anual('Ventas', 'Crecimiento Ventas')
agregar_fila(df_anual_filtrado, 'Cost of Goods Sold', 'Costo de Ventas')
agregar_crecimiento_anual('Costo de Ventas', 'Crecimiento Costo de Ventas')
agregar_fila(df_anual_filtrado, 'Total Inventories', 'Inventarios')
agregar_crecimiento_anual('Inventarios', 'Crecimiento Inventarios')
agregar_razon_financiera(df_anual_filtrado, '  Accounts Receivable', 'Cuentas por cobrar/ventas', 'Revenue', como_porcentaje=True)
agregar_fila(df_anual_filtrado, 'Dividends per Share', 'Dividendos por Accion')
agregar_fila(df_anual_filtrado, 'Free Cash Flow', 'Flujo de Caja Libre')
dividir_filas_razones_financieras('Flujo de Caja Libre', 'Ventas', 'FCF/ventas',True)
dividir_filas_razones_financieras('Flujo de Caja Libre','# de Acciones','Free cash flow per share',True)
agregar_razon_financiera(df_anual_filtrado,'Free Cash Flow','FCF/DEBT','Short-Term Debt & Capital Lease Obligation','Long-Term Debt & Capital Lease Obligation',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'Free Cash Flow','FCF/DEBT corto plazo','Short-Term Debt & Capital Lease Obligation',como_porcentaje=False)
dividir_filas_razones_financieras('Dividendos por Accion',"EPS",'Dividend Payout Ratio',True)
agregar_razon_financiera(df_anual_filtrado,'Net Income','Net Income vs Deuda Largo Plazo','Long-Term Debt & Capital Lease Obligation',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'Total Liabilities','Pasivo/Fondos Propios','Total Equity',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'Total Current Assets','Ratio de Solvencia','Total Current Liabilities',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'Long-Term Debt & Capital Lease Obligation','Deuda/Capital','Total Equity',como_porcentaje=False)
agregar_razon_financiera(df_anual_filtrado,'  Interest Expense','Gasto financiero','Operating Income',como_porcentaje=True) ##Interes Expense viene negativo, tenerlo en cuenta ya que si queda positivo seria malo ya que el operating income es tambien negativo
agregar_razon_financiera(df_anual_filtrado,'Total Inventories','Inventory to current Assetis','Total Current Assets',como_porcentaje=True)
agregar_fila(df_anual_filtrado,'Cash Conversion Cycle','Net Trading Cycle')
agregar_fila(df_anual_filtrado,'PEG Ratio','PEGY Ratio')
agregar_fila(df_anual_filtrado,'PE Ratio','PER')
agregar_fila(df_anual_filtrado,'PS Ratio','P/S Ratio')
agregar_razon_financiera(df_anual_filtrado,'  Accounts Receivable','Receivable to Current Assets','Total Current Assets',como_porcentaje=True)
agregar_fila(df_anual_filtrado,'ROA %','ROA%')
agregar_fila(df_anual_filtrado,'ROE %','ROE%')
agregar_razon_financiera(df_anual_filtrado,'Total Assets','ROB','Total Equity',como_porcentaje=False)
agregar_fila(df_anual_filtrado,'ROIC %','ROIC %')
agregar_fila(df_anual_filtrado,'WACC %','WACC %')
agregar_razon_financiera_numerador(df_anual_filtrado,'Test de Acidez','Total Current Liabilities','Total Current Assets','Total Inventories',como_porcentaje=False)
agregar_fila(df_anual_filtrado,'Book Value per Share','Valor en libros')
agregar_fila(df_anual_filtrado,'Piotroski F-Score','Piotrivski F-Score')
agregar_fila(df_anual_filtrado,'Altman Z-Score','Altman Z score')
agregar_fila(df_anual_filtrado,'Beneish M-Score','Beneish M-Score')
agregar_fila(df_anual_filtrado,'Dividends per Share','Dividendo por accion')
agregar_crecimiento_anual('Dividendo por accion','Crecimiento Dividendo por accion')
razones_financieras

,Dec2005,Dec2006,Dec2007,Dec2008,Dec2009,Dec2010,Dec2011,Dec2012,Dec2013,Dec2014,Dec2015,Dec2016,Dec2017,Dec2018,Dec2019,Dec2020,Dec2021,Dec2022,Dec2023
Amortizacion(%),7.04,6.70,6.95,7.73,6.27,5.81,5.41,5.47,5.67,5.79,5.96,6.56,7.02,6.23,8.30,12.53,9.12,7.59,6.91
Beneficio Neto(%),9.28,9.99,10.01,8.82,8.06,9.88,11.16,10.87,10.39,9.56,10.09,10.10,8.09,8.69,8.63,0.65,8.54,9.90,10.00
Beneficio Neto,2835.542,3597.488,3831.502,3528.434,3786.754,5154.685,6543.365,7197.7,7701.93,7763.48,9210.729,10140.432,9885.69,11704.347,12383.12,750.115,12868.176,17384.903,19486.518
Margen Bruto(%),33.91,33.62,33.60,34.39,40.97,40.01,40.45,40.33,40.44,40.66,40.69,40.16,39.93,39.42,39.50,36.03,37.66,38.76,40.01
Margen Operativo(%),6.38,8.93,9.40,7.01,11.70,14.81,17.33,16.91,14.59,13.68,16.01,13.35,14.99,15.05,16.08,3.31,12.43,14.53,15.20
EPS,2.11,2.68,2.85,2.63,2.82,3.84,4.88,5.36,5.74,5.78,6.86,7.56,7.37,8.72,9.25,0.56,9.56,12.95,14.52
Revenue Per Share,22.77,26.84,28.46,29.82,35.01,38.86,43.70,49.36,55.21,60.51,68.02,74.83,91.02,100.39,107.16,85.96,111.95,130.88,145.14
# de Acciones,1342.196,1342.346,1344.387,1341.61,1342.743,1342.196,1342.196,1342.196,1342.196,1342.196,1342.196,1342.196,1342.196,1342.196,1339.323,1338.502,1346.254,1342.206,1342.196
Ventas,30556.905,36022.457,38267.248,40004.117,47003.657,52160.681,58656.809,66246.504,74105.444,81213.589,91292.889,100441.536,122168.279,134739.99,143527.524,115058.886,150709.751,175663.359,194812.281
Crecimiento Beneficio Neto,0.00,26.87,6.50,-7.91,7.32,36.12,26.94,10.00,7.01,0.80,18.64,10.09,-2.51,18.40,5.80,-93.94,1615.49,35.10,12.09


In [901]:
razones_financieras.loc["Beneish M-Score"] = razones_financieras.loc["Beneish M-Score"].astype(float).abs()
razones_financieras.loc["Gasto financiero"] = razones_financieras.loc["Gasto financiero"].astype(float).abs()

In [902]:
# Convertir los valores a numéricos, convirtiendo los no numéricos a NaN
razones_financieras_numeric = razones_financieras.apply(pd.to_numeric, errors='coerce')

# Ignorar las columnas que contienen "2020" o "2021" para las filas específicas
filas_a_ignorar = ["Crecimiento Beneficio Neto", "Crecimiento Costo de Ventas", "Crecimiento Inventarios", "Crecimiento Ventas", "Crecimiento Dividendo por accion"]

# Crear una copia del DataFrame para modificar
razones_financieras_modificado = razones_financieras_numeric.copy()

# Establecer los valores de las columnas a ignorar en NaN para las filas específicas
for fila in filas_a_ignorar:
    for col in razones_financieras_modificado.columns:
        if '2020' in col or '2021' in col:
            razones_financieras_modificado.loc[fila, col] = np.nan

# Calcular el promedio de cada fila, ignorando los NaN y valores no finitos
promedios = razones_financieras_modificado.apply(lambda row: np.nanmean(row[np.isfinite(row)]), axis=1)

# Redondear los valores a dos decimales
promedios_redondeados = promedios.round(2)

# Convertir los promedios a un DataFrame
promedios_df = promedios_redondeados.to_frame(name='Promedio')
promedios_df

,Promedio
Amortizacion(%),7.00
Beneficio Neto(%),9.09
Beneficio Neto,8197.63
Margen Bruto(%),38.45
Margen Operativo(%),12.72
EPS,6.11
Revenue Per Share,68.73
# de Acciones,1342.19
Ventas,92244.84
Crecimiento Beneficio Neto,12.43


## Dividend Discount Model

$$
P_0 = \frac{D_1}{r - g}
$$

In [903]:
# Cargar datos del stock
data_stock = pd.read_csv(archivo_stock + '.csv')
data_stock['Fecha'] = pd.to_datetime(data_stock['Fecha'], format='%d.%m.%Y')
data_stock['Cierre'] = data_stock['Cierre'].astype(str).str.replace(',', '').astype(float)
data_stock = data_stock.sort_values('Fecha')

# Filtrar los últimos 5 años
ultima_fecha_2023 = data_stock[data_stock['Fecha'].dt.year == 2023]['Fecha'].max()
fecha_actual_stock = ultima_fecha_2023
fecha_inicio_stock = fecha_actual_stock - pd.DateOffset(years=5)
datos_x_anos_stock = data_stock[data_stock['Fecha'] >= fecha_inicio_stock]

# Calcular el crecimiento basado en precios
precio_inicial_stock = datos_x_anos_stock.iloc[0]['Cierre']
precio_final_stock = datos_x_anos_stock.iloc[-1]['Cierre']


n = 5  # número de años
crecimiento_stock = ((precio_final_stock / precio_inicial_stock) ** (1 / n)) - 1
crecimiento_stock_porcentaje = crecimiento_stock * 100

# Sumar los dividendos de los últimos 5 años
ultimas_5_columnas = razones_financieras.columns[-n:]
dividendos_ultimos_5_anios = razones_financieras.loc["Dividendos por Accion", ultimas_5_columnas]
suma_dividendos_ultimos_5_anios = dividendos_ultimos_5_anios.sum()

# Calcular el crecimiento total ajustado por dividendos
precio_final_stock_ajustado = precio_final_stock + suma_dividendos_ultimos_5_anios
crecimiento_stock_irt = ((precio_final_stock_ajustado / precio_inicial_stock) ** (1 / n)) - 1
crecimiento_stock_irt_porcentaje = crecimiento_stock_irt * 100

print(f"5-Year Growth Stock: {crecimiento_stock_porcentaje:.2f}%")
print(f"5-Year Growth Stock IRT: {crecimiento_stock_irt_porcentaje:.2f}%")


5-Year Growth Stock: -3.15%
5-Year Growth Stock IRT: -1.59%


In [904]:


df_beta
promedio_cetes
crecimiento_ipc_porcentaje_años
df_crecimiento_ipc = pd.DataFrame(crecimiento_ipc_porcentaje_años)[['año_cierre', 'CAGR_%']]

# Renombrar columnas si deseas algo más claro
df_crecimiento_ipc.columns = ['Año_Cierre', 'Crecimiento_IPC_%5_años']
df_crecimiento_ipc


,Año_Cierre,Crecimiento_IPC_%5_años
0,2023,0.0998
1,2022,0.0225
2,2021,0.0558
3,2020,0.0286
4,2019,0.0335
5,2018,0.0185
6,2017,0.0400
7,2016,0.0595
8,2015,0.0430
9,2014,0.0350


In [905]:
# Asegurar que las columnas clave para el merge sean iguales
df_crecimiento_ipc.columns = ['Año', 'Crecimiento_IPC']
df_beta.columns = ['Año', 'Beta']
promedio_cetes.columns = ['Año', 'CETES']

# Unir los tres DataFrames por año
df_r = df_crecimiento_ipc.merge(df_beta, on='Año') \
                         .merge(promedio_cetes, on='Año')

# Calcular 'r' usando la fórmula
df_r['r'] = df_r['CETES'] + df_r['Beta'] * (df_r['Crecimiento_IPC'] - df_r['CETES'])

# Mostrar resultado
print(df_r)


     Año  Crecimiento_IPC    Beta     CETES         r
0   2023           0.0998  0.7428  0.114710  0.103635
1   2022           0.0225  0.3051  0.092586  0.071203
2   2021           0.0558  0.5047  0.052381  0.054107
3   2020           0.0286  1.3802  0.052056  0.019682
4   2019           0.0335  1.0586  0.078175  0.030882
5   2018           0.0185  1.0291  0.080373  0.016699
6   2017           0.0400  1.6133  0.070847  0.021082
7   2016           0.0595  0.8511  0.046900  0.057624
8   2015           0.0430  0.7197  0.034980  0.040752
9   2014           0.0350  0.7742  0.032781  0.034499
10  2013           0.0327  0.8611  0.039327  0.033620


In [906]:
crecimiento_dividendo_ultimas_3 = razones_financieras.loc["Crecimiento Dividendo por accion"]

crecimiento_dividendo_ultimas_3 = crecimiento_dividendo_ultimas_3.loc[~crecimiento_dividendo_ultimas_3.index.str.contains("2021")]

crecimiento_dividendo_ultimas_3.index = crecimiento_dividendo_ultimas_3.index.str.extract(r'(\d{4})').astype(int)[0]

crecimiento_dividendo_ultimas_3 = crecimiento_dividendo_ultimas_3.to_frame(name="Crecimiento_Dividendo")
crecimiento_dividendo_ultimas_3["Crecimiento_Dividendo"] = pd.to_numeric(crecimiento_dividendo_ultimas_3["Crecimiento_Dividendo"], errors="coerce")

crecimiento_dividendo_ultimas_3["Crecimiento_Dividendo"] = pd.to_numeric(
    crecimiento_dividendo_ultimas_3["Crecimiento_Dividendo"], errors="coerce"
)

# Reemplazar valores inf y -inf por NaN (este paso es clave)
crecimiento_dividendo_ultimas_3["Crecimiento_Dividendo"] = crecimiento_dividendo_ultimas_3["Crecimiento_Dividendo"].replace([np.inf, -np.inf], 0)

# (Opcional) Reemplazar ceros si también los quieres excluir

crecimiento_dividendo_ultimas_3["Crecimiento_Dividendo"] = crecimiento_dividendo_ultimas_3["Crecimiento_Dividendo"].replace([np.inf, -np.inf], 0)

crecimiento_dividendo_ultimas_3

,Crecimiento_Dividendo
0,
2005,0.00
2006,0.00
2007,17.65
2008,10.00
2009,-9.09
2010,10.00
2011,22.73
2012,24.07
2013,8.96


In [907]:
promedios = {}
for año in range(2014, 2024):
    ultimos_3 = crecimiento_dividendo_ultimas_3.loc[(crecimiento_dividendo_ultimas_3.index < año)].tail(3)
    promedio = ultimos_3["Crecimiento_Dividendo"].mean()
    promedios[año] = promedio

# Crear DataFrame con resultados si lo necesitas más adelante
df_promedios_div = pd.DataFrame.from_dict(promedios, orient='index', columns=['Promedio_Crecimiento_Dividendo'])
df_promedios_div.index.name = 'Año'
df_promedios_div = df_promedios_div.reset_index()

df_promedios_div

,Año,Promedio_Crecimiento_Dividendo
0,2014,18.586667
1,2015,-22.323333
2,2016,-30.346667
3,2017,-27.160000
4,2018,6.173333
5,2019,6.173333
6,2020,4.860000
7,2021,-1.200000
8,2022,-1.200000
9,2023,1.763333


In [908]:

dividendos = razones_financieras.loc["Dividendos por Accion"]

# Convertir índice a años numéricos
dividendos.index = dividendos.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a DataFrame
dividendos = dividendos.to_frame(name="Dividendos_por_Accion")

# Convertir a tipo numérico
dividendos["Dividendos_por_Accion"] = pd.to_numeric(dividendos["Dividendos_por_Accion"], errors="coerce")

# Filtrar solo de 2019 a 2023
dividendos = dividendos.loc[(dividendos.index >= 2013) & (dividendos.index <= 2023)]

# Resetear índice si lo necesitas como columna
dividendos = dividendos.reset_index().rename(columns={0: "Año"})

dividendos

,Año,Dividendos_por_Accion
0,2013,0.73
1,2014,0.00
2,2015,0.81
3,2016,0.96
4,2017,0.96
5,2018,0.96
6,2019,1.10
7,2020,0.90
8,2021,2.25
9,2022,2.45


In [909]:
# Asegúrate de tener los años como índices en ambos DataFrames
dividendos.set_index('Año', inplace=True)
df_promedios_div.set_index('Año', inplace=True)

# Calcular D1 directamente usando broadcast
d1 = dividendos['Dividendos_por_Accion'] * (1 + df_promedios_div['Promedio_Crecimiento_Dividendo'] / 100)

# Si quieres volverlo DataFrame
df_d1 = d1.to_frame(name='D1').reset_index()

df_d1

,Año,D1
0,2013,NaN
1,2014,0.000000
2,2015,0.629181
3,2016,0.668672
4,2017,0.699264
5,2018,1.019264
6,2019,1.167907
7,2020,0.943740
8,2021,2.223000
9,2022,2.420600


In [910]:
df_d1
df_promedios_div
df_r

,Año,Crecimiento_IPC,Beta,CETES,r
0,2023,0.0998,0.7428,0.114710,0.103635
1,2022,0.0225,0.3051,0.092586,0.071203
2,2021,0.0558,0.5047,0.052381,0.054107
3,2020,0.0286,1.3802,0.052056,0.019682
4,2019,0.0335,1.0586,0.078175,0.030882
5,2018,0.0185,1.0291,0.080373,0.016699
6,2017,0.0400,1.6133,0.070847,0.021082
7,2016,0.0595,0.8511,0.046900,0.057624
8,2015,0.0430,0.7197,0.034980,0.040752
9,2014,0.0350,0.7742,0.032781,0.034499


In [911]:
df_d1

,Año,D1
0,2013,NaN
1,2014,0.000000
2,2015,0.629181
3,2016,0.668672
4,2017,0.699264
5,2018,1.019264
6,2019,1.167907
7,2020,0.943740
8,2021,2.223000
9,2022,2.420600


In [912]:
df_promedios_div

,Promedio_Crecimiento_Dividendo
Año,
2014,18.586667
2015,-22.323333
2016,-30.346667
2017,-27.160000
2018,6.173333
2019,6.173333
2020,4.860000
2021,-1.200000
2022,-1.200000


In [913]:
df_d1 = df_d1.sort_values("Año").reset_index(drop=True)
df_promedios_div = df_promedios_div.sort_values("Año").reset_index(drop=True)
df_r = df_r.sort_values("Año").reset_index(drop=True)

# Extraer columnas necesarias
d1 = df_d1["D1"]
g = df_promedios_div["Promedio_Crecimiento_Dividendo"] / 100  # convertir a decimal
r = df_r["r"]

# Calcular P0
denominador = r - g
p0 = d1 / denominador

# Reemplazar valores inválidos (cuando r <= g)
p0[denominador <= 0] = None

# Crear DataFrame con resultados
df_p0 = pd.DataFrame({
    "Año": df_d1["Año"],
    "P0": p0
})

print(df_p0)

     Año         P0
0   2013        NaN
1   2014   0.000000
2   2015   1.827853
3   2016   2.031056
4   2017        NaN
5   2018        NaN
6   2019        NaN
7   2020  29.787865
8   2021  33.627520
9   2022  45.186221
10  2023        NaN


## Book Value per Share

In [914]:
total_assets = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Assets'].iloc[0, 1:]
total_liabilities = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]
total_shares = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Shares Outstanding (Basic Average)'].iloc[0, 1:]

In [915]:
# Calcular el valor en libros (Book Value) para cada año
book_value = total_assets - total_liabilities

# Calcular el valor en libros por acción (Book Value per Share) para cada año
book_value_per_share = book_value / total_shares

# Crear un DataFrame para almacenar los resultados
book_value_per_share_anual = pd.DataFrame({
    'Book Value': book_value,
    'Book Value per Share': book_value_per_share
}, index=total_assets.index)

book_value_per_share_anual

,Book Value,Book Value per Share
0,,
Dec2005,20066.972,14.950851
Dec2006,23813.973,17.740562
Dec2007,27184.22,20.220532
Dec2008,30186.968,22.500554
Dec2009,33336.656,24.82728
Dec2010,37580.67,27.999391
Dec2011,43258.989,32.230009
Dec2012,49529.71,36.901995
Dec2013,54827.332,40.848976


## Liquidation Value


Utilizable con empresas a la baja, si el precio se encuentra por debajo de este,podria ser una oportunidad

1️⃣ Ratio de Recuperación del Efectivo (
𝑅
𝐸
R 
E
​
 )
(Normalmente 100%)
Se incluyen todas las cuentas que representan efectivo o equivalentes de efectivo, que son altamente líquidos.

Cuentas relevantes:

Cash And Cash Equivalents
Marketable Securities
Cash, Cash Equivalents, Marketable Securities (si está consolidado)
2️⃣ Ratio de Recuperación de Cuentas por Cobrar (
𝑅
𝐶
R 
C
​
 )
(Usualmente entre 70-90%)
Se incluyen todas las cuentas por cobrar, notas por cobrar y otros montos adeudados a la empresa.

Cuentas relevantes:

Accounts Receivable
Notes Receivable
Loans Receivable
Other Current Receivables
Total Receivables (si está consolidado)
3️⃣ Ratio de Recuperación de Inventarios (
𝑅
𝐼
R 
I
​
 )
(Suele ser 50-80%)
Incluye todos los bienes en inventario que la empresa posee, como materia prima, productos en proceso y productos terminados.

Cuentas relevantes:

Inventories, Raw Materials & Components
Inventories, Work In Process
Inventories, Finished Goods
Inventories, Other
Total Inventories (si está consolidado)
4️⃣ Ratio de Recuperación de Propiedades, Planta y Equipos (
𝑅
𝑃
R 
P
​
 )
(Usualmente 60-90%)
Incluye activos fijos tangibles como edificios, maquinaria y terrenos.

Cuentas relevantes:

Land And Improvements
Buildings And Improvements
Machinery, Furniture, Equipment
Construction In Progress
Other Gross PPE
Gross Property, Plant and Equipment
Property, Plant and Equipment (si está consolidado)
⚠️ Acumulated Depreciation → Se resta porque representa la pérdida de valor de los activos.
5️⃣ Ratio de Recuperación de Otros Activos Tangibles (
𝑅
𝑂
R 
O
​
 )
(Depende del tipo de activo, entre 10-80%)
Incluye otros activos físicos que no encajan en las categorías anteriores, como infraestructura, vehículos y mobiliario.

Cuentas relevantes:

Investments And Advances (si contiene activos tangibles)
Other Long Term Assets (si son tangibles)
Otros activos que no sean intangibles o Goodwill
⚠️ No se incluyen:

Intangible Assets (valor de marca, patentes, etc.)
Goodwill (ya que en liquidación normalmente no tiene valor)

In [916]:
ratio_recuperacion_efectivo = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Cash, Cash Equivalents, Marketable Securities'].iloc[0, 1:] * 0.95

ratio_recuperacion_cc = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Receivables'].iloc[0, 1:] * 0.75

ratio_recuperacion_inv = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Inventories'].iloc[0, 1:] * 0.60

ratio_recuperacion_ppe = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Property, Plant and Equipment'].iloc[0, 1:] * 0.70

ratio_recuperacion_act_tangibles =df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Investments And Advances'].iloc[0, 1:] + df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Other Long Term Assets'].iloc[0, 1:] *.30

total_liabilities = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]


In [917]:
liquidation_value = ratio_recuperacion_efectivo + ratio_recuperacion_cc + ratio_recuperacion_inv + ratio_recuperacion_ppe + ratio_recuperacion_act_tangibles - total_liabilities

liquidation_value_per_share = liquidation_value / total_shares

liquidation_value_per_share

0
Dec2005     8.196721
Dec2006      8.85126
Dec2007     9.775396
Dec2008     8.703412
Dec2009     4.017587
Dec2010    10.372747
Dec2011     9.896419
Dec2012     9.610001
Dec2013      9.76289
Dec2014    13.615529
Dec2015    17.044141
Dec2016    16.565765
Dec2017     9.395932
Dec2018    14.276961
Dec2019    15.085006
Dec2020    14.028601
Dec2021    19.972074
Dec2022    22.629431
Dec2023    28.804731
dtype: object

## EPV

In [918]:
# Extraer la serie del margen operativo
margen_operativo = razones_financieras.loc["Margen Operativo(%)"].copy()

# Convertir índice a años numéricos
margen_operativo.index = margen_operativo.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a DataFrame y valores numéricos
df_margen = margen_operativo.to_frame(name="Margen_Operativo")
df_margen["Margen_Operativo"] = pd.to_numeric(df_margen["Margen_Operativo"], errors="coerce")

# Asegurar índice correcto
df_margen = df_margen.sort_index()
df_margen.index = df_margen.index.astype(int)

# Promedio móvil de 5 años hacia atrás (excluye año actual)
serie_promedios = (
    df_margen["Margen_Operativo"]
        .shift(1)                          # excluye el año actual
        .rolling(window=5, min_periods=5)
        .mean()
)

# Quedarse solo con los últimos 10 años disponibles
serie_promedios = serie_promedios.dropna().tail(11)

# Convertir a DataFrame con el mismo formato original
df_promedios_ebit = (
    serie_promedios
        .round(2)
        .div(100)                          # convertir a decimal
        .to_frame(name="Promedio_Margen_Operativo")
        .reset_index()
        .rename(columns={"index": "Año"})
)

df_promedios_ebit

,0,Promedio_Margen_Operativo
0,2013,0.1355
1,2014,0.1507
2,2015,0.1546
3,2016,0.1570
4,2017,0.1491
5,2018,0.1452
6,2019,0.1462
7,2020,0.1510
8,2021,0.1256
9,2022,0.1237


In [919]:
##Obtener las ventas de promedio de los ultimos 10 años

# Extraer la serie de ventas
ventas = razones_financieras.loc["Ventas"].copy()

# Convertir índice a años numéricos
ventas.index = ventas.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a DataFrame y a valores numéricos
df_ventas = ventas.to_frame(name="Ventas")
df_ventas["Ventas"] = pd.to_numeric(df_ventas["Ventas"], errors="coerce")

# Asegurar índice correcto
df_ventas = df_ventas.sort_index()
df_ventas.index = df_ventas.index.astype(int)

# Promedio móvil de 5 años hacia atrás (excluye año actual)
serie_promedios = (
    df_ventas["Ventas"]
        .shift(1)                          # excluye el año actual
        .rolling(window=5, min_periods=5)
        .mean()
)

# Quedarse solo con los últimos 10 años disponibles
serie_promedios = serie_promedios.dropna().tail(11)

# Crear DataFrame final (mismos nombres que el original)
df_promedios_ventas = (
    serie_promedios
        .round(2)
        .to_frame(name="Promedio_Ventas")
        .reset_index()
        .rename(columns={"index": "Año"})
)

df_promedios_ventas


,0,Promedio_Ventas
0,2013,52814.35
1,2014,59634.62
2,2015,66476.61
3,2016,74303.05
4,2017,82659.99
5,2018,93844.35
6,2019,105971.26
7,2020,118434.04
8,2021,123187.24
9,2022,133240.89


In [920]:

SGyA = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Selling, General, & Admin. Expense'].iloc[0, 1:]
# Convertir índices trimestrales a fechas
SGyA.index = pd.to_datetime(SGyA.index, format="%b%Y")

# Crear DataFrame a partir de la Serie
df_sgya = SGyA.astype(str).str.replace(",", "").astype(float).to_frame(name="SGyA")

# Crear una columna con el año
df_sgya["Año"] = df_sgya.index.year
anios_completos = (
    df_sgya.groupby("Año")
           .size()
           .loc[lambda s: s >= 4]
           .index
           .tolist()
)

ultimos_10_anios = sorted(anios_completos)[-11:]
# Calcular el promedio de los últimos 20 trimestres antes de cada año de cierre
promedios_sgya = {}
for año in ultimos_10_anios:
    # Tomar datos anteriores al año actual
    datos_previos = df_sgya[df_sgya.index < pd.to_datetime(f"{año}-01-01")].tail(20)
    promedio = round((datos_previos["SGyA"] * 0.75).mean(), 2)
    promedios_sgya[año] = promedio

# Crear DataFrame con resultados
df_promedios_sgya = pd.DataFrame.from_dict(promedios_sgya, orient="index", columns=["Promedio_SG&A_ajustado"])
df_promedios_sgya.index.name = "Año"
df_promedios_sgya = df_promedios_sgya.reset_index()
df_promedios_sgya

,Año,Promedio_SG&A_ajustado
0,2013,713.49
1,2014,1335.12
2,2015,2057.37
3,2016,2660.42
4,2017,3512.32
5,2018,4278.13
6,2019,4587.22
7,2020,5822.41
8,2021,6299.29
9,2022,7353.73


In [921]:
# Asegurar que la primera columna se llame "Año"
df_promedios_ventas = df_promedios_ventas.rename(
    columns={df_promedios_ventas.columns[0]: "Año"}
).sort_values("Año").reset_index(drop=True)

df_promedios_ebit = df_promedios_ebit.rename(
    columns={df_promedios_ebit.columns[0]: "Año"}
).sort_values("Año").reset_index(drop=True)

df_promedios_sgya = df_promedios_sgya.rename(
    columns={df_promedios_sgya.columns[0]: "Año"}
).sort_values("Año").reset_index(drop=True)


# Extraer columnas
ventas = df_promedios_ventas["Promedio_Ventas"]
margen = df_promedios_ebit["Promedio_Margen_Operativo"]
sgya = df_promedios_sgya["Promedio_SG&A_ajustado"]

# Calcular EBIT normalizado
ebit_normalizado = round((ventas * margen) + sgya, 2)

# Crear DataFrame final
df_ebit_normalizado = pd.DataFrame({
    "Año": df_promedios_ventas["Año"],
    "EBIT_normalizado": ebit_normalizado
})

df_ebit_normalizado

,Año,EBIT_normalizado
0,2013,7869.83
1,2014,10322.06
2,2015,12334.65
3,2016,14326.00
4,2017,15836.92
5,2018,17904.33
6,2019,20080.22
7,2020,23705.95
8,2021,21771.61
9,2022,23835.63


In [922]:
# Extraer la fila de Tax Rate %
tax_rate = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Tax Rate %'].iloc[0, 1:]

# Convertir índice a años
tax_rate.index = tax_rate.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a DataFrame y a numérico
df_tax = tax_rate.to_frame(name="Tax_Rate")
df_tax["Tax_Rate"] = pd.to_numeric(df_tax["Tax_Rate"], errors="coerce")

# Calcular promedio móvil de 5 años positivos antes de cada año de 2019 a 2023
# Obtener los últimos 10 años disponibles
ultimos_10_anios = sorted(df_tax.index.unique())[-11:]

# Calcular promedio móvil de 5 años positivos antes de cada año
promedios_tax = {}
for año in ultimos_10_anios:
    ultimos_5 = df_tax[df_tax.index < año].tail(5)
    positivos = ultimos_5[ultimos_5["Tax_Rate"] >= 0]
    promedio = round(positivos["Tax_Rate"].mean(), 2) / 100
    promedios_tax[año] = promedio

# Crear DataFrame con resultados
df_promedios_tax = pd.DataFrame.from_dict(promedios_tax, orient='index', columns=['Promedio_Tasa_Impuestos'])
df_promedios_tax.index.name = 'Año'
df_promedios_tax = df_promedios_tax.reset_index()

df_promedios_tax

,Año,Promedio_Tasa_Impuestos
0,2013,0.2908
1,2014,0.2830
2,2015,0.2741
3,2016,0.2656
4,2017,0.2662
5,2018,0.2573
6,2019,0.2567
7,2020,0.2579
8,2021,0.2570
9,2022,0.2434


In [923]:
# Asegurar el mismo orden por año
df_ebit_normalizado = df_ebit_normalizado.sort_values("Año").reset_index(drop=True)
df_promedios_tax = df_promedios_tax.sort_values("Año").reset_index(drop=True)

# Extraer columnas
ebit = df_ebit_normalizado["EBIT_normalizado"]
tax = df_promedios_tax["Promedio_Tasa_Impuestos"]

# Calcular After-Tax EBIT
after_tax_ebit = ebit * (1 - tax)

# Crear DataFrame con resultados
df_after_tax_ebit = pd.DataFrame({
    "Año": df_ebit_normalizado["Año"],
    "After_Tax_EBIT": after_tax_ebit.round(2)
})

print(df_after_tax_ebit)

     Año  After_Tax_EBIT
0   2013         5581.28
1   2014         7400.92
2   2015         8953.72
3   2016        10521.01
4   2017        11621.13
5   2018        13297.55
6   2019        14925.63
7   2020        17592.19
8   2021        16176.31
9   2022        18034.04
10  2023        19852.97


In [924]:
##Promedio depreciacion
# Extraer la fila de DDA
DDA = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Depreciation, Depletion and Amortization'].iloc[0, 1:]

# Convertir índice a años
DDA.index = DDA.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a DataFrame y asegurar valores numéricos
df_dda = DDA.to_frame(name="DDA")
df_dda["DDA"] = pd.to_numeric(df_dda["DDA"], errors="coerce")

# Obtener los últimos 10 años disponibles
ultimos_10_anios = sorted(df_dda.index.unique())[-11:]

# Calcular promedio móvil de 5 años antes de cada año
promedios_dda = {}
for año in ultimos_10_anios:
    ultimos_5 = df_dda[df_dda.index < año].tail(5)
    promedio = round(ultimos_5["DDA"].mean(), 2)
    promedios_dda[año] = promedio

# Crear DataFrame con resultados
df_promedios_dda = pd.DataFrame.from_dict(promedios_dda, orient='index', columns=['Promedio_DDA'])
df_promedios_dda.index.name = 'Año'
df_promedios_dda = df_promedios_dda.reset_index()

df_promedios_dda

,Año,Promedio_DDA
0,2013,1245.95
1,2014,1373.38
2,2015,1513.89
3,2016,1714.26
4,2017,1986.61
5,2018,2378.65
6,2019,2700.86
7,2020,3259.88
8,2021,3855.70
9,2022,4361.87


In [925]:
# Asegurar orden por año
df_promedios_dda = df_promedios_dda.sort_values("Año").reset_index(drop=True)
df_promedios_tax = df_promedios_tax.sort_values("Año").reset_index(drop=True)

# Extraer columnas
dda = df_promedios_dda["Promedio_DDA"]
tax = df_promedios_tax["Promedio_Tasa_Impuestos"]

# Calcular Depreciación en exceso
depreciacion_exceso = dda * 0.5 * tax

# Crear DataFrame con resultados
df_depreciacion_exceso = pd.DataFrame({
    "Año": df_promedios_dda["Año"],
    "Depreciacion_en_exceso": depreciacion_exceso.round(2)
})

print(df_depreciacion_exceso)

     Año  Depreciacion_en_exceso
0   2013                  181.16
1   2014                  194.33
2   2015                  207.48
3   2016                  227.65
4   2017                  264.42
5   2018                  306.01
6   2019                  346.66
7   2020                  420.36
8   2021                  495.46
9   2022                  530.84
10  2023                  583.75


In [926]:
# Asegurar orden por año
df_after_tax_ebit = df_after_tax_ebit.sort_values("Año").reset_index(drop=True)
df_depreciacion_exceso = df_depreciacion_exceso.sort_values("Año").reset_index(drop=True)

# Extraer columnas
after_tax = df_after_tax_ebit["After_Tax_EBIT"]
dep_exceso = df_depreciacion_exceso["Depreciacion_en_exceso"]

# Calcular earnings normalizados
normalized_earnings = after_tax + dep_exceso

# Crear DataFrame final
df_normalized_earnings = pd.DataFrame({
    "Año": df_after_tax_ebit["Año"],
    "Normalized_earnings": normalized_earnings.round(2)
})

print(df_normalized_earnings)

     Año  Normalized_earnings
0   2013              5762.44
1   2014              7595.25
2   2015              9161.20
3   2016             10748.66
4   2017             11885.55
5   2018             13603.56
6   2019             15272.29
7   2020             18012.55
8   2021             16671.77
9   2022             18564.88
10  2023             20436.72


In [927]:
accumulated_depreciation = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == '  Accumulated Depreciation'].iloc[0, 1:]
gppe = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Gross Property, Plant and Equipment'].iloc[0, 1:]

# Limpiar y convertir índices a años
accumulated_depreciation.index = accumulated_depreciation.index.str.extract(r'(\d{4})').astype(int)[0]
gppe.index = gppe.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a numérico
accumulated_depreciation = pd.to_numeric(accumulated_depreciation, errors='coerce').abs()
gppe = pd.to_numeric(gppe, errors='coerce')

# Unir en DataFrame
df_ppe = pd.DataFrame({
    "Gross_PPE": gppe,
    "Accumulated_Depreciation": accumulated_depreciation
})

# Calcular Net PPE
df_ppe["Net_PPE"] = df_ppe["Gross_PPE"] - df_ppe["Accumulated_Depreciation"]

# Tomar los últimos 10 años disponibles
ultimos_10_anios = sorted(df_ppe.index)[-11:]

df_net_ppe = df_ppe.loc[ultimos_10_anios].copy()
df_net_ppe["Año"] = df_net_ppe.index
df_net_ppe = df_net_ppe.reset_index(drop=True)

df_net_ppe

,Gross_PPE,Accumulated_Depreciation,Net_PPE,Año
0,41393.708,12339.445,29054.263,2013
1,44035.598,13645.315,30390.283,2014
2,45314.659,13389.836,31924.823,2015
3,50271.909,14808.398,35463.511,2016
4,60954.055,17098.178,43855.877,2017
5,66421.088,19305.984,47115.104,2018
6,83730.734,21641.179,62089.555,2019
7,86521.028,24178.147,62342.881,2020
8,90772.341,26685.165,64087.176,2021
9,97131.217,29224.402,67906.815,2022


In [928]:
# Extraer las filas correspondientes
capex = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Capital Expenditure'].iloc[0, 1:]
revenue = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]

# Limpiar índices a años
capex.index = capex.index.str.extract(r'(\d{4})').astype(int)[0]
revenue.index = revenue.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a valores numéricos
capex = pd.to_numeric(capex, errors='coerce').abs()
revenue = pd.to_numeric(revenue, errors='coerce')

# Obtener los últimos 10 años comunes entre capex y revenue
anios_comunes = sorted(capex.index.intersection(revenue.index))[-16:]

capex = capex.loc[anios_comunes]
revenue = revenue.loc[anios_comunes]

# Unir en un DataFrame
df_capex_revenue = pd.DataFrame({
    "Año": capex.index,
    "Capital_Expenditure": capex.values,
    "Revenue": revenue.values
})

df_capex_revenue

,Año,Capital_Expenditure,Revenue
0,2008,3864.930,40004.117
1,2009,1139.369,47003.657
2,2010,2617.067,52160.681
3,2011,4152.262,58656.809
4,2012,6114.645,66246.504
5,2013,4397.802,74105.444
6,2014,3320.103,81213.589
7,2015,3706.227,91292.889
8,2016,6601.959,100441.536
9,2017,6859.204,122168.279


In [929]:
# Asegurar que el DataFrame esté ordenado por año
df_capex_revenue = df_capex_revenue.sort_values("Año").reset_index(drop=True)

# Calcular variación porcentual año a año del Revenue
df_capex_revenue["Crecimiento_Revenue"] = df_capex_revenue["Revenue"].pct_change()

# Obtener los últimos 10 años disponibles
ultimos_10_anios = sorted(df_capex_revenue["Año"].unique())[-11:]

# Calcular el promedio de los 5 años anteriores para cada año
promedios_crecimiento = {}
for año in ultimos_10_anios:
    fila = df_capex_revenue[df_capex_revenue["Año"].between(año - 5, año - 1)]
    promedio = fila["Crecimiento_Revenue"].mean()
    promedios_crecimiento[año] = round(promedio, 4)

# Convertir a DataFrame
df_promedio_crecimiento_revenue = pd.DataFrame.from_dict(
    promedios_crecimiento, orient='index', columns=["Promedio_Crecimiento_Revenue"]
).reset_index().rename(columns={"index": "Año"})

df_promedio_crecimiento_revenue


,Año,Promedio_Crecimiento_Revenue
0,2013,0.1347
1,2014,0.1314
2,2015,0.1156
3,2016,0.1185
4,2017,0.1137
5,2018,0.1310
6,2019,0.1279
7,2020,0.1218
8,2021,0.0573
9,2022,0.0992


In [930]:
# Asegurar orden por año en todos los DataFrames
df_net_ppe = df_net_ppe.sort_values("Año").reset_index(drop=True)
df_capex_revenue = df_capex_revenue.sort_values("Año").reset_index(drop=True)
df_promedio_crecimiento_revenue = df_promedio_crecimiento_revenue.sort_values("Año").reset_index(drop=True)

df_inputs = df_capex_revenue.merge(df_net_ppe[["Año", "Net_PPE"]], on="Año")
df_inputs = df_inputs.merge(df_promedio_crecimiento_revenue, on="Año")
ultimos_10_anios = sorted(df_inputs["Año"].unique())[-11:]
df_inputs = df_inputs[df_inputs["Año"].isin(ultimos_10_anios)].copy()

# 1. Calcular growth_capex solo si hay crecimiento positivo
df_inputs["Growth_Capex"] = (df_inputs["Net_PPE"] / df_inputs["Revenue"]) * \
                            (df_inputs["Promedio_Crecimiento_Revenue"] * df_inputs["Revenue"])
df_inputs["Growth_Capex"] = df_inputs["Growth_Capex"].where(df_inputs["Promedio_Crecimiento_Revenue"] > 0, 0)

# 2. Calcular mantenimiento preliminar
df_inputs["Maintenance_Capex"] = df_inputs["Capital_Expenditure"] - df_inputs["Growth_Capex"]

# 3. Si crecimiento fue negativo o resultado fue negativo, se usa todo el CAPEX
df_inputs["Maintenance_Capex"] = df_inputs["Maintenance_Capex"].where(
    (df_inputs["Promedio_Crecimiento_Revenue"] > 0) & (df_inputs["Maintenance_Capex"] >= 0),
    df_inputs["Capital_Expenditure"]
)

# Resultado por año
df_maintenance_capex = df_inputs[["Año", "Maintenance_Capex"]].copy()

# Promedio final si lo necesitas
maintenance_capex_promedio = round(df_maintenance_capex["Maintenance_Capex"].mean(), 2)

print(df_maintenance_capex)

     Año  Maintenance_Capex
0   2013         484.192774
1   2014        3320.103000
2   2015          15.717461
3   2016        2399.532947
4   2017        1872.790785
5   2018         364.459376
6   2019        6560.982000
7   2020        4365.648000
8   2021        1661.935815
9   2022        6330.741000
10  2023        2062.532427


WACC=( 
V
E
​
 ×r 
e
​
 )+( 
V
D
​
 ×r 
d
​
 ×(1−t))
 

In [931]:
# Convertir el índice a años
total_shares.index = total_shares.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a DataFrame
df_total_shares = total_shares.to_frame(name="Total_Shares")

# Convertir valores a numérico
df_total_shares["Total_Shares"] = pd.to_numeric(df_total_shares["Total_Shares"], errors="coerce")

# Agregar la columna 'Año' desde el índice
df_total_shares["Año"] = df_total_shares.index

ultimos_10_anios = sorted(df_total_shares.index)[-11:]
df_total_shares = df_total_shares.loc[ultimos_10_anios].reset_index(drop=True)

df_total_shares

,Total_Shares,Año
0,1342.196,2013
1,1342.196,2014
2,1342.196,2015
3,1342.196,2016
4,1342.196,2017
5,1342.196,2018
6,1339.323,2019
7,1338.502,2020
8,1346.254,2021
9,1342.206,2022


In [932]:
# Asegurar que resumen_cierre tenga la columna 'Año' correctamente
resumen_cierre["Año"] = resumen_cierre["Año"].astype(int)

# Filtrar solo años 2019–2023
ultimos_10_anios = sorted(resumen_cierre["Año"].unique())[-11:]
resumen_cierre_filtrado = resumen_cierre[resumen_cierre["Año"].isin(ultimos_10_anios)].copy()
df_total_shares = df_total_shares.sort_values("Año").reset_index(drop=True)

# Calcular el precio mínimo entre promedio y mediana
precio_accion = resumen_cierre_filtrado[["Promedio_Cierre", "Mediana_Cierre"]].min(axis=1)

# Calcular E
df_equity = pd.DataFrame({
    "Año": df_total_shares["Año"],
    "Precio_Accion_Min": precio_accion,
    "Total_Shares": df_total_shares["Total_Shares"],
    "E": df_total_shares["Total_Shares"] * precio_accion
})

df_equity["E"] = df_equity["E"].round(2)
df_equity


,Año,Precio_Accion_Min,Total_Shares,E
0,2013,146.881538,1342.196,197143.81
1,2014,146.560962,1342.196,196713.54
2,2015,185.550000,1342.196,249044.47
3,2016,191.624615,1342.196,257197.79
4,2017,142.654340,1342.196,191470.08
5,2018,130.687308,1342.196,175407.98
6,2019,107.695000,1339.323,144238.39
7,2020,60.770000,1338.502,81340.77
8,2021,82.280577,1346.254,110770.56
9,2022,98.625000,1342.206,132375.07


In [933]:
Deuda_corto_plazo = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Short-Term Debt & Capital Lease Obligation'].iloc[0, 1:]
Deuda_largo_plazo = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Long-Term Debt & Capital Lease Obligation'].iloc[0, 1:]

# --- Deuda corto plazo ---
Deuda_corto_plazo.index = Deuda_corto_plazo.index.str.extract(r'(\d{4})').astype(int)[0]
df_deuda_cp = Deuda_corto_plazo.to_frame(name="Deuda_Corto_Plazo")
df_deuda_cp["Deuda_Corto_Plazo"] = pd.to_numeric(df_deuda_cp["Deuda_Corto_Plazo"], errors="coerce")
df_deuda_cp["Año"] = df_deuda_cp.index  # <-- esta línea es clave
df_deuda_cp = df_deuda_cp.loc[ultimos_10_anios].reset_index(drop=True)

# --- Deuda largo plazo ---
Deuda_largo_plazo.index = Deuda_largo_plazo.index.str.extract(r'(\d{4})').astype(int)[0]
df_deuda_lp = Deuda_largo_plazo.to_frame(name="Deuda_Largo_Plazo")
df_deuda_lp["Deuda_Largo_Plazo"] = pd.to_numeric(df_deuda_lp["Deuda_Largo_Plazo"], errors="coerce")
df_deuda_lp["Año"] = df_deuda_lp.index  # <-- también clave aquí
df_deuda_lp = df_deuda_lp.loc[ultimos_10_anios].reset_index(drop=True)

# Ahora sí puedes ordenarlos y sumarlos
df_deuda_cp = df_deuda_cp.sort_values("Año").reset_index(drop=True)
df_deuda_lp = df_deuda_lp.sort_values("Año").reset_index(drop=True)

# Calcular deuda total (D)
df_deuda_total = pd.DataFrame({
    "Año": df_deuda_cp["Año"],
    "Deuda_Total": df_deuda_cp["Deuda_Corto_Plazo"] + df_deuda_lp["Deuda_Largo_Plazo"]
})

df_deuda_total


,Año,Deuda_Total
0,2013,14932.584
1,2014,13343.876
2,2015,14096.066
3,2016,30571.826
4,2017,36217.501
5,2018,31088.067
6,2019,47538.484
7,2020,50965.036
8,2021,46493.311
9,2022,43324.078


In [934]:
market_risk_premium = 0.06
df_r["Cost_of_Equity"] = df_r["CETES"] + df_r["Beta"] * market_risk_premium

# Redondear si deseas
df_r["Cost_of_Equity"] = df_r["Cost_of_Equity"].round(5)

df_r

,Año,Crecimiento_IPC,Beta,CETES,r,Cost_of_Equity
0,2013,0.0327,0.8611,0.039327,0.033620,0.09099
1,2014,0.0350,0.7742,0.032781,0.034499,0.07923
2,2015,0.0430,0.7197,0.034980,0.040752,0.07816
3,2016,0.0595,0.8511,0.046900,0.057624,0.09797
4,2017,0.0400,1.6133,0.070847,0.021082,0.16764
5,2018,0.0185,1.0291,0.080373,0.016699,0.14212
6,2019,0.0335,1.0586,0.078175,0.030882,0.14169
7,2020,0.0286,1.3802,0.052056,0.019682,0.13487
8,2021,0.0558,0.5047,0.052381,0.054107,0.08266
9,2022,0.0225,0.3051,0.092586,0.071203,0.11089


In [935]:
interest_expense = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == '  Interest Expense'].iloc[0, 1:].abs()

# Convertir el índice a años numéricos
interest_expense.index = interest_expense.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a DataFrame
df_interest_expense = interest_expense.to_frame(name="Interest_Expense")
df_interest_expense["Año"] = df_interest_expense.index
# Asegurar que los valores sean numéricos
df_interest_expense["Interest_Expense"] = pd.to_numeric(df_interest_expense["Interest_Expense"], errors="coerce")

# Filtrar solo de 2019 a 2023
df_interest_expense = (
    df_interest_expense
        .loc[ultimos_10_anios]
        .reset_index()
        .rename(columns={"index": "Año"})
)
df_interest_expense

,0,Interest_Expense,Año
0,2013,1088.892,2013
1,2014,1164.789,2014
2,2015,970.015,2015
3,2016,1344.225,2016
4,2017,2770.722,2017
5,2018,2695.911,2018
6,2019,3678.467,2019
7,2020,4210.487,2020
8,2021,4649.854,2021
9,2022,3989.142,2022


In [936]:
# Asegurar orden y resetear índices
df_interest_expense = df_interest_expense.sort_values("Año").reset_index(drop=True)
df_deuda_total = df_deuda_total.sort_values("Año").reset_index(drop=True)

# Calcular rd
df_cost_of_debt = pd.DataFrame({
    "Año": df_interest_expense["Año"],
    "Cost_of_Debt": df_interest_expense["Interest_Expense"] / df_deuda_total["Deuda_Total"]
})

# Redondear a 5 decimales si deseas
df_cost_of_debt["Cost_of_Debt"] = df_cost_of_debt["Cost_of_Debt"].round(5)

df_cost_of_debt

,Año,Cost_of_Debt
0,2013,0.07292
1,2014,0.08729
2,2015,0.06881
3,2016,0.04397
4,2017,0.07650
5,2018,0.08672
6,2019,0.07738
7,2020,0.08262
8,2021,0.10001
9,2022,0.09208


In [937]:
# Asegurarte de que el índice sea numérico (por si acaso)
df_tax.index = df_tax.index.astype(int)

# Mover el índice a una columna 'Año'
df_tax["Año"] = df_tax.index

# Reiniciar índice y ordenar
df_tax = df_tax.reset_index(drop=True).sort_values("Año")

# Asegurarte de que 'Tax_Rate' sea numérico y convertir de porcentaje a decimal
df_tax["Tax_Rate"] = pd.to_numeric(df_tax["Tax_Rate"], errors="coerce") / 100

In [938]:
# Asegurar que todos los DataFrames estén alineados por Año y ordenados
df_equity = df_equity.sort_values("Año").reset_index(drop=True)
df_deuda_total = df_deuda_total.sort_values("Año").reset_index(drop=True)
df_r = df_r.sort_values("Año").reset_index(drop=True)
df_cost_of_debt = df_cost_of_debt.sort_values("Año").reset_index(drop=True)
df_tax = df_tax.sort_values("Año").reset_index(drop=True)

# Extraer columnas necesarias
E = df_equity["E"]
D = df_deuda_total["Deuda_Total"]
cost_of_equity = df_r["Cost_of_Equity"]
rd = df_cost_of_debt["Cost_of_Debt"]
tax_rate = df_tax["Tax_Rate"]

# Calcular WACC
wacc = (E / (E + D)) * cost_of_equity + (D / (E + D)) * rd * (1 - tax_rate)

# Crear DataFrame final
df_wacc = pd.DataFrame({
    "Año": df_equity["Año"],
    "WACC": wacc.round(5)
})

df_wacc = df_wacc.dropna().reset_index(drop=True)
df_wacc

,Año,WACC
0,2013.0,0.09084
1,2014.0,0.07830
2,2015.0,0.07660
3,2016.0,0.09351
4,2017.0,0.14949
5,2018.0,0.12992
6,2019.0,0.13083
7,2020.0,0.10594
8,2021.0,0.08012
9,2022.0,0.10024


In [940]:
# Asegurar orden y alineación
df_normalized_earnings = df_normalized_earnings.sort_values("Año").reset_index(drop=True)
df_maintenance_capex = df_maintenance_capex.sort_values("Año").reset_index(drop=True)
df_wacc = df_wacc.sort_values("Año").reset_index(drop=True)

# Extraer columnas
earnings = df_normalized_earnings["Normalized_earnings"]
maintenance = df_maintenance_capex["Maintenance_Capex"]
wacc = df_wacc["WACC"]

# Calcular EPV
epv_business_operation = (earnings - maintenance.mean()) / wacc

# Resultado en DataFrame
df_epv = pd.DataFrame({
    "Año": df_normalized_earnings["Año"],
    "EPV_Business_Operation": epv_business_operation.round(2)
})

print(df_epv)

     Año  EPV_Business_Operation
0   2013                33974.02
1   2014                62822.61
2   2015                84660.06
3   2016                86326.81
4   2017                61604.86
5   2018                84108.07
6   2019                96278.00
7   2020               144764.12
8   2021               174682.11
9   2022               158505.99
10  2023               127333.53


In [941]:
cash_and_equivalents = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Cash, Cash Equivalents, Marketable Securities'].iloc[0, 1:]
# Extraer el año desde el índice
cash_and_equivalents.index = cash_and_equivalents.index.str.extract(r'(\d{4})').astype(int)[0]

# Convertir a DataFrame
df_cash = cash_and_equivalents.to_frame(name="Cash_and_Equivalents")

# Asegurar valores numéricos
df_cash["Cash_and_Equivalents"] = pd.to_numeric(df_cash["Cash_and_Equivalents"], errors="coerce")

# Crear columna 'Año' desde el índice
df_cash["Año"] = df_cash.index

# Opcional: filtrar solo años 2019 a 2023
df_cash = df_cash[df_cash["Año"].isin(ultimos_10_anios)].reset_index(drop=True)
df_cash


,Cash_and_Equivalents,Año
0,1618.060,2013
1,5891.468,2014
2,8583.219,2015
3,25574.230,2016
4,16635.078,2017
5,13535.499,2018
6,18634.798,2019
7,26195.936,2020
8,32494.873,2021
9,24516.254,2022


In [942]:
# Asegurar que todos los DataFrames estén alineados por año
df_epv = df_epv.sort_values("Año").reset_index(drop=True)
df_cash = df_cash.sort_values("Año").reset_index(drop=True)
df_deuda_total = df_deuda_total.sort_values("Año").reset_index(drop=True)
df_total_shares = df_total_shares.sort_values("Año").reset_index(drop=True)

# Extraer columnas
epv_ops = df_epv["EPV_Business_Operation"]
cash = df_cash["Cash_and_Equivalents"]
deuda = df_deuda_total["Deuda_Total"]
shares = df_total_shares["Total_Shares"]

# Calcular EPV final por acción
epv_final = (epv_ops + cash - deuda) / shares

# Crear DataFrame final
df_epv_final = pd.DataFrame({
    "Año": df_epv["Año"],
    "EPV_Final_Por_Accion": epv_final.round(2)
})

print(df_epv_final)

     Año  EPV_Final_Por_Accion
0   2013                 15.39
1   2014                 41.25
2   2015                 58.97
3   2016                 60.59
4   2017                 31.31
5   2018                 49.59
6   2019                 50.30
7   2020                 89.65
8   2021                119.36
9   2022                104.08
10  2023                 86.18


## NET CURRENT ASSET VALUE

Mas usado para empresas con altos valores de current assets

In [ ]:
total_current_assets = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Current Assets'].iloc[0, 1:]
total_liabilities = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]
minority_interest = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Minority Interest'].iloc[0, 1:]
prefered_stock = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Preferred Stock'].iloc[0, 1:]


In [ ]:
# --- Extraer y limpiar índices ---
total_current_assets.index = total_current_assets.index.str.extract(r'(\d{4})').astype(int)[0]
total_liabilities.index = total_liabilities.index.str.extract(r'(\d{4})').astype(int)[0]
minority_interest.index = minority_interest.index.str.extract(r'(\d{4})').astype(int)[0]
prefered_stock.index = prefered_stock.index.str.extract(r'(\d{4})').astype(int)[0]

# --- Convertir a numérico ---
total_current_assets = pd.to_numeric(total_current_assets, errors='coerce')
total_liabilities = pd.to_numeric(total_liabilities, errors='coerce')
minority_interest = pd.to_numeric(minority_interest, errors='coerce')
prefered_stock = pd.to_numeric(prefered_stock, errors='coerce')

# --- Unir en DataFrame ---
df_balance_resumen = pd.DataFrame({
    "Total_Current_Assets": total_current_assets,
    "Total_Liabilities": total_liabilities,
    "Minority_Interest": minority_interest,
    "Preferred_Stock": prefered_stock
})

# --- Agregar columna 'Año' y resetear índice ---
df_balance_resumen["Año"] = df_balance_resumen.index
df_balance_resumen = df_balance_resumen.reset_index(drop=True)

# --- Opcional: filtrar años 2019–2023 ---
df_balance_resumen = df_balance_resumen[df_balance_resumen["Año"].between(2019, 2023)]
df_balance_resumen

,Total_Current_Assets,Total_Liabilities,Minority_Interest,Preferred_Stock,Año
6,77082.460,91487.331,239.686,0,2019
7,80920.042,97023.338,243.382,0,2020
8,92345.612,102083.154,259.494,0,2021
9,94546.550,103420.442,271.734,0,2022
10,111121.479,111654.452,287.367,0,2023


In [ ]:
# Calcular Net Current Asset Value
df_balance_resumen["Net_Current_Asset_Value"] = (
    df_balance_resumen["Total_Current_Assets"]
    - df_balance_resumen["Total_Liabilities"]
    - df_balance_resumen["Minority_Interest"]
    - df_balance_resumen["Preferred_Stock"]
).round(2)

# Mostrar resultado
print(df_balance_resumen[["Año", "Net_Current_Asset_Value"]])

     Año  Net_Current_Asset_Value
6   2019                -14644.56
7   2020                -16346.68
8   2021                 -9997.04
9   2022                 -9145.63
10  2023                  -820.34


In [ ]:
# Asegurar orden
df_balance_resumen = df_balance_resumen.sort_values("Año").reset_index(drop=True)
df_total_shares = df_total_shares.sort_values("Año").reset_index(drop=True)

# Calcular NCAV por acción
ncav_per_share = df_balance_resumen["Net_Current_Asset_Value"] / df_total_shares["Total_Shares"]

# Crear nuevo DataFrame
df_ncav_per_share = pd.DataFrame({
    "Año": df_balance_resumen["Año"],
    "NCAV_Per_Share": ncav_per_share.round(2)
})

print(df_ncav_per_share)

    Año  NCAV_Per_Share
0  2019          -10.93
1  2020          -12.21
2  2021           -7.43
3  2022           -6.81
4  2023           -0.61


## Tangicle Book Value

In [ ]:
total_equity = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Stockholders Equity'].iloc[0, 1:]
prefered_stock = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Preferred Stock'].iloc[0, 1:]
intagible_assets = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Intangible Assets'].iloc[0, 1:]

# --- Convertir índices a años numéricos ---
total_equity.index = total_equity.index.str.extract(r'(\d{4})').astype(int)[0]
prefered_stock.index = prefered_stock.index.str.extract(r'(\d{4})').astype(int)[0]
intagible_assets.index = intagible_assets.index.str.extract(r'(\d{4})').astype(int)[0]

# --- Convertir a numérico ---
total_equity = pd.to_numeric(total_equity, errors='coerce')
prefered_stock = pd.to_numeric(prefered_stock, errors='coerce')
intagible_assets = pd.to_numeric(intagible_assets, errors='coerce')

# --- Unir en un solo DataFrame ---
df_equity_base = pd.DataFrame({
    "Total_Equity": total_equity,
    "Preferred_Stock": prefered_stock,
    "Intangible_Assets": intagible_assets
})

# --- Agregar columna 'Año' y resetear índice ---
df_equity_base["Año"] = df_equity_base.index
df_equity_base = df_equity_base.reset_index(drop=True)

# --- Opcional: filtrar años 2019–2023 ---
df_equity_base = df_equity_base[df_equity_base["Año"].between(2019, 2023)]

df_equity_base

,Total_Equity,Preferred_Stock,Intangible_Assets,Año
6,108834.852,0,16175.038,2019
7,107576.649,0,15900.027,2020
8,119628.025,0,15880.069,2021
9,132182.782,0,15534.602,2022
10,147212.352,0,15612.080,2023


In [ ]:
# Calcular Tangible Book Value
df_equity_base["Tangible_Book_Value"] = (
    df_equity_base["Total_Equity"] -
    df_equity_base["Preferred_Stock"] -
    df_equity_base["Intangible_Assets"]
).round(2)

df_equity_base

,Total_Equity,Preferred_Stock,Intangible_Assets,Año,Tangible_Book_Value
6,108834.852,0,16175.038,2019,92659.81
7,107576.649,0,15900.027,2020,91676.62
8,119628.025,0,15880.069,2021,103747.96
9,132182.782,0,15534.602,2022,116648.18
10,147212.352,0,15612.080,2023,131600.27


In [ ]:
# Asegurar orden y reinicio de índice
df_equity_base = df_equity_base.sort_values("Año").reset_index(drop=True)
df_total_shares = df_total_shares.sort_values("Año").reset_index(drop=True)

# Calcular valor por acción
valor_por_accion = df_equity_base["Tangible_Book_Value"] / df_total_shares["Total_Shares"]

# Crear nuevo DataFrame
df_tangible_book_value_per_share = pd.DataFrame({
    "Año": df_equity_base["Año"],
    "Tangible_Book_Value_Per_Share": valor_por_accion.round(2)
})

df_tangible_book_value_per_share

,Año,Tangible_Book_Value_Per_Share
0,2019,69.18
1,2020,68.49
2,2021,77.06
3,2022,86.91
4,2023,98.05


## Projected Free Cash Flow

In [ ]:
ebitda = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'EBITDA'].iloc[0, 1:]
free_cash_flow = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Free Cash Flow'].iloc[0, 1:]
revenue = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]

# --- Limpiar índice a años ---
ebitda.index = ebitda.index.str.extract(r'(\d{4})').astype(int)[0]
free_cash_flow.index = free_cash_flow.index.str.extract(r'(\d{4})').astype(int)[0]

# --- Convertir a DataFrames y a valores numéricos ---
df_ebitda = ebitda.to_frame(name="EBITDA")
df_ebitda["EBITDA"] = pd.to_numeric(df_ebitda["EBITDA"], errors="coerce")
df_ebitda["Año"] = df_ebitda.index
df_ebitda = df_ebitda.reset_index(drop=True)

df_fcf = free_cash_flow.to_frame(name="Free_Cash_Flow")
df_fcf["Free_Cash_Flow"] = pd.to_numeric(df_fcf["Free_Cash_Flow"], errors="coerce")
df_fcf["Año"] = df_fcf.index
df_fcf = df_fcf.reset_index(drop=True)

revenue.index = revenue.index.str.extract(r'(\d{4})').astype(int)[0]

# --- Convertir a DataFrame y a valores numéricos ---
df_revenue = revenue.to_frame(name="Revenue")
df_revenue["Revenue"] = pd.to_numeric(df_revenue["Revenue"], errors="coerce")
df_revenue["Año"] = df_revenue.index
df_revenue = df_revenue.reset_index(drop=True)

In [ ]:
def crecimiento_5_anios(df, columna):
    df = df.sort_values("Año").reset_index(drop=True)
    resultados = []

    for i in range(5, len(df)):
        año = df.loc[i, "Año"]
        datos_previos = df.loc[i-5:i-1, columna]
        crecimiento = datos_previos.pct_change().mean()
        resultados.append({"Año": año, f"{columna}_crecimiento": round(crecimiento, 3)})

    return pd.DataFrame(resultados)

df_ebitda_crecimiento = crecimiento_5_anios(df_ebitda, "EBITDA")
df_fcf_crecimiento = crecimiento_5_anios(df_fcf, "Free_Cash_Flow")
df_revenue_crecimiento = crecimiento_5_anios(df_revenue, "Revenue")

# Unir los tres DataFrames por 'Año'
df_crecimientos = df_ebitda_crecimiento \
    .merge(df_fcf_crecimiento, on="Año") \
    .merge(df_revenue_crecimiento, on="Año")

# Mostrar resultado
print(df_crecimientos)


    Año  EBITDA_crecimiento  Free_Cash_Flow_crecimiento  Revenue_crecimiento
0  2018               0.098                       0.700                0.134
1  2019               0.124                       0.037                0.136
2  2020               0.129                       0.132                0.121
3  2021              -0.060                       0.176                0.047
4  2022               0.343                       0.377                0.070
5  2023               0.367                       0.200                0.086


In [ ]:
def calcular_cagr_por_año(df, columna):
    df = df.sort_values("Año").reset_index(drop=True)
    resultados = []

    for i in range(5, len(df)):
        año_final = df.loc[i, "Año"]
        valor_inicial = df.loc[i - 5, columna]
        valor_final = df.loc[i, columna]

        if valor_inicial > 0 and valor_final > 0:
            cagr = (valor_final / valor_inicial) ** (1/5) - 1
            resultados.append({"Año": año_final, f"CAGR_{columna}": round(cagr, 3)})

    return pd.DataFrame(resultados)

df_cagr_ebitda = calcular_cagr_por_año(df_ebitda, "EBITDA")
df_cagr_fcf = calcular_cagr_por_año(df_fcf, "Free_Cash_Flow")
df_cagr_revenue = calcular_cagr_por_año(df_revenue, "Revenue")

df_cagr_todo = df_cagr_ebitda \
    .merge(df_cagr_fcf, on="Año") \
    .merge(df_cagr_revenue, on="Año")

print(df_cagr_todo)

    Año  CAGR_EBITDA  CAGR_Free_Cash_Flow  CAGR_Revenue
0  2018        0.105                0.320         0.127
1  2019        0.132                0.007         0.121
2  2020       -0.090                0.147         0.047
3  2021        0.080                0.297         0.085
4  2022        0.112                0.137         0.075
5  2023        0.109                0.120         0.077


In [ ]:
df_crecimientos_final = df_crecimientos.merge(df_cagr_todo, on="Año")

# Definir columnas numéricas a evaluar
columnas_crecimiento = [col for col in df_crecimientos_final.columns if col != "Año"]

# Calcular el valor mínimo válido por fila
valores_filtrados = []
for _, fila in df_crecimientos_final.iterrows():
    valores_validos = [
        fila[col] for col in columnas_crecimiento
        if isinstance(fila[col], (int, float)) and 0.04 <= fila[col] <= 0.11
    ]
    valor_final = min(valores_validos) if valores_validos else 0.05
    valores_filtrados.append(round(valor_final, 3))

# Crear DataFrame final
df_min_crecimiento = pd.DataFrame({
    "Año": df_crecimientos_final["Año"],
    "Crecimiento_Seleccionado": valores_filtrados
})

print(df_min_crecimiento)


    Año  Crecimiento_Seleccionado
0  2018                     0.098
1  2019                     0.050
2  2020                     0.047
3  2021                     0.047
4  2022                     0.070
5  2023                     0.077


In [ ]:
# Calcular growth_assumption y growth_multiple
df_min_crecimiento["Growth_Assumption"] = df_min_crecimiento["Crecimiento_Seleccionado"] * 100
df_min_crecimiento["Growth_Multiple"] = 8.3459 * (1.07 ** (df_min_crecimiento["Growth_Assumption"] - 4))

# Redondear si deseas
df_min_crecimiento["Growth_Assumption"] = df_min_crecimiento["Growth_Assumption"].round(2)
df_min_crecimiento["Growth_Multiple"] = df_min_crecimiento["Growth_Multiple"].round(2)

print(df_min_crecimiento)

    Año  Crecimiento_Seleccionado  Growth_Assumption  Growth_Multiple
0  2018                     0.098                9.8            12.36
1  2019                     0.050                5.0             8.93
2  2020                     0.047                4.7             8.75
3  2021                     0.047                4.7             8.75
4  2022                     0.070                7.0            10.22
5  2023                     0.077                7.7            10.72


In [ ]:
# Convertir a DataFrame
df_fcf = free_cash_flow.to_frame(name="Free_Cash_Flow")

# Asegurar que los valores sean numéricos
df_fcf["Free_Cash_Flow"] = pd.to_numeric(df_fcf["Free_Cash_Flow"], errors="coerce")

# Crear columna 'Año' desde el índice (que ya es numérico)
df_fcf["Año"] = df_fcf.index

# Resetear índice
df_fcf = df_fcf.reset_index(drop=True)

df_fcf

,Free_Cash_Flow,Año
0,2108.661,2013
1,8335.453,2014
2,5362.473,2015
3,5654.017,2016
4,6501.238,2017
5,8460.244,2018
6,8640.036,2019
7,10656.604,2020
8,20788.799,2021
9,12346.421,2022


In [ ]:
# Asegurar orden por año
df_fcf = df_fcf.sort_values("Año").reset_index(drop=True)

# Lista para guardar resultados
promedios_fcf = []

# Calcular promedio de los 5 años anteriores para cada año
for i in range(5, len(df_fcf)):
    año = df_fcf.loc[i, "Año"]
    valores_previos = df_fcf.loc[i-5:i-1, "Free_Cash_Flow"]
    promedio = round(valores_previos.mean(), 2)
    promedios_fcf.append({"Año": año, "FCF_5y_Promedio": promedio})

# Convertir a DataFrame
df_fcf_5y_promedio = pd.DataFrame(promedios_fcf)

print(df_fcf_5y_promedio)


    Año  FCF_5y_Promedio
0  2018          5592.37
1  2019          6862.68
2  2020          6923.60
3  2021          7982.43
4  2022         11009.38
5  2023         12178.42


In [ ]:
free_cash_flow_4trimestres = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Free Cash Flow'].iloc[0, 1:]

# Paso 1: Asegurar que la Serie tenga su índice correctamente
free_cash_flow_4trimestres.index.name = "Periodo"  # Darle nombre al índice para manipularlo

# Paso 2: Convertir a DataFrame
df_fcf_trim = free_cash_flow_4trimestres.to_frame(name="FCF_4T").reset_index()

# Paso 3: Extraer el año del índice 'Periodo'
df_fcf_trim["Año"] = df_fcf_trim["Periodo"].str.extract(r'(\d{4})').astype(int)

# Paso 4: Convertir valores a numérico
df_fcf_trim["FCF_4T"] = pd.to_numeric(df_fcf_trim["FCF_4T"], errors="coerce")

# Paso 5: Agrupar por año y obtener promedio
df_fcf_anual_promedio = df_fcf_trim.groupby("Año")["FCF_4T"].mean().reset_index()
df_fcf_anual_promedio = df_fcf_anual_promedio.rename(columns={"FCF_4T": "FCF_Promedio_Anual_4T"})
df_fcf_anual_promedio["FCF_Promedio_Anual_4T"] = df_fcf_anual_promedio["FCF_Promedio_Anual_4T"].round(2)

print(df_fcf_anual_promedio)



     Año  FCF_Promedio_Anual_4T
0   2014                2083.86
1   2015                1340.62
2   2016                1413.50
3   2017                1625.31
4   2018                2115.06
5   2019                2160.01
6   2020                2664.15
7   2021                5197.20
8   2022                3086.61
9   2023                3723.73
10  2024               -1244.12


In [ ]:
# Asegurar que el índice sea string y contenga el año
total_stockholders_equity = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Total Stockholders Equity'].iloc[0, 1:]

total_stockholders_equity.index.name = "Periodo"

# Convertir a DataFrame
df_total_equity = total_stockholders_equity.to_frame(name="Total_Stockholders_Equity").reset_index()

# Extraer el año de 'Periodo'
df_total_equity["Año"] = df_total_equity["Periodo"].astype(str).str.extract(r'(\d{4})').astype(int)

# Convertir valores a numérico
df_total_equity["Total_Stockholders_Equity"] = pd.to_numeric(
    df_total_equity["Total_Stockholders_Equity"], errors="coerce"
)

# Limpiar el DataFrame final
df_total_equity = df_total_equity[["Año", "Total_Stockholders_Equity"]]

print(df_total_equity)

     Año  Total_Stockholders_Equity
0   2013                  54824.983
1   2014                  62663.243
2   2015                  71339.556
3   2016                  81331.751
4   2017                  89858.091
5   2018                 100469.837
6   2019                 108834.852
7   2020                 107576.649
8   2021                 119628.025
9   2022                 132182.782
10  2023                 147212.352


In [ ]:
# Asegurar que ambas tablas estén ordenadas y listas
df_fcf_5y_promedio = df_fcf_5y_promedio.sort_values("Año").reset_index(drop=True)
df_fcf_anual_promedio = df_fcf_anual_promedio.sort_values("Año").reset_index(drop=True)

# Hacer merge solo donde haya coincidencia de años
df_fcf_ajustado = df_fcf_5y_promedio.merge(df_fcf_anual_promedio, on="Año", how="inner")

# Aplicar la fórmula
df_fcf_ajustado["FCF_Ajustado"] = (
    (6 * df_fcf_ajustado["FCF_5y_Promedio"] + 0.75 * df_fcf_ajustado["FCF_Promedio_Anual_4T"]) / 6.75
).round(2)

print(df_fcf_ajustado[["Año", "FCF_Ajustado"]])

    Año  FCF_Ajustado
0  2018       5206.00
1  2019       6340.16
2  2020       6450.33
3  2021       7672.96
4  2022      10129.07
5  2023      11239.01


In [ ]:
# Asegurarte de que los valores estén en formato decimal
df_resultados_inf_mexico["Tasa de Inflación"] = pd.to_numeric(
    df_resultados_inf_mexico["Tasa de Inflación"], errors="coerce"
)

# Calcular el factor de inflación a 3 años para cada año
df_resultados_inf_mexico["Factor_Inflacion"] = (
    (1 + df_resultados_inf_mexico["Tasa de Inflación"]) ** 3
).round(4)

print(df_resultados_inf_mexico)


     Año  Tasa de Inflación  Factor_Inflacion
0   2023             0.0466            1.1464
1   2022             0.0782            1.2534
2   2021             0.0736            1.2374
3   2020             0.0315            1.0975
4   2019             0.0283            1.0873
5   2018             0.0483            1.1520
6   2017             0.0677            1.2172
7   2016             0.0336            1.1042
8   2015             0.0213            1.0653
9   2014             0.0408            1.1275
10  2013             0.0397            1.1239


In [ ]:
# Asegurar que ambos DataFrames están ordenados por año
df_fcf_ajustado = df_fcf_ajustado.sort_values("Año").reset_index(drop=True)
df_resultados_inf_mexico = df_resultados_inf_mexico.sort_values("Año").reset_index(drop=True)

# Hacer merge solo donde haya coincidencia de años
df_fcf_ajustado_inflacion = df_fcf_ajustado.merge(
    df_resultados_inf_mexico[["Año", "Factor_Inflacion"]],
    on="Año",
    how="inner"
)

# Calcular FCF ajustado con inflación
df_fcf_ajustado_inflacion["FCF_Ajustado_Inflacion"] = (
    df_fcf_ajustado_inflacion["FCF_Ajustado"] * df_fcf_ajustado_inflacion["Factor_Inflacion"]
).round(2)

# Mostrar resultado final
print(df_fcf_ajustado_inflacion[["Año", "FCF_Ajustado_Inflacion"]])


    Año  FCF_Ajustado_Inflacion
0  2018                 5997.31
1  2019                 6893.66
2  2020                 7079.24
3  2021                 9494.52
4  2022                12695.78
5  2023                12884.40


In [ ]:
# Asegurar que todos estén ordenados y con índice reiniciado
df_min_crecimiento = df_min_crecimiento.sort_values("Año").reset_index(drop=True)
df_fcf_ajustado_inflacion = df_fcf_ajustado_inflacion.sort_values("Año").reset_index(drop=True)
df_total_equity = df_total_equity.sort_values("Año").reset_index(drop=True)
df_total_shares = df_total_shares.sort_values("Año").reset_index(drop=True)

# Unir todos por año
df_intrinsic = df_min_crecimiento.merge(
    df_fcf_ajustado_inflacion[["Año", "FCF_Ajustado_Inflacion"]], on="Año", how="inner"
).merge(
    df_total_equity[["Año", "Total_Stockholders_Equity"]], on="Año", how="inner"
).merge(
    df_total_shares[["Año", "Total_Shares"]], on="Año", how="inner"
)

# Aplicar fórmula
df_intrinsic["Intrinsic_Value"] = (
    (df_intrinsic["Growth_Multiple"] * df_intrinsic["FCF_Ajustado_Inflacion"] +
     0.7 * df_intrinsic["Total_Stockholders_Equity"]) /
    df_intrinsic["Total_Shares"]
).round(2)

# Mostrar resultado final
print(df_intrinsic[["Año", "Intrinsic_Value"]])


    Año  Intrinsic_Value
0  2019           102.85
1  2020           102.54
2  2021           123.91
3  2022           165.61
4  2023           179.68


## Median PS value

In [ ]:

# --- Revenue ---
revenue_serie = df_anual[df_anual['Fiscal Period'] == 'Revenue'].iloc[0, 1:]
revenue_serie.index = revenue_serie.index.astype(str)

revenue_serie = pd.to_numeric(revenue_serie, errors='coerce').dropna()

revenue_10_anios = revenue_serie[-22:-1]
revenue_10_anios.index = [int(str(i)[-4:]) for i in revenue_10_anios.index]


# --- Shares ---
shares_serie = df_anual[
    df_anual['Fiscal Period'] == 'Shares Outstanding (Basic Average)'
].iloc[0, 1:]
shares_serie.index = shares_serie.index.astype(str)

shares_serie = pd.to_numeric(shares_serie, errors='coerce').dropna()

shares_10_anios = shares_serie[-22:-1]
shares_10_anios.index = [int(str(i)[-4:]) for i in shares_10_anios.index]

revenue_per_share = revenue_10_anios / shares_10_anios


In [ ]:
revenue_per_share

2005     22.766351
2006     26.835449
2007     28.464459
2008     29.817993
2009     35.005699
2010     38.862194
2011     43.702119
2012     49.356803
2013     55.212088
2014     60.507995
2015     68.017554
2016     74.833732
2017     91.021191
2018    100.387715
2019    107.164234
2020     85.960937
2021    111.947486
2022    130.876601
2023    145.144436
dtype: float64

In [ ]:
archivo_stock_10_anios = 'Datos históricos LIVEPOLC1_20anios'

data_stock = pd.read_csv(archivo_stock_10_anios + '.csv')

# Convertir las fechas y ordenar
data_stock['Fecha'] = pd.to_datetime(data_stock['Fecha'], format='%d.%m.%Y')

data_stock = data_stock.sort_values('Fecha')

# Convertir la columna 'Cierre' a tipo numérico, eliminando caracteres no válidos
data_stock['Cierre'] = data_stock['Cierre'].astype(str).str.replace(',', '').astype(float)

# Verificar valores nulos y eliminarlos
data_stock = data_stock.dropna(subset=['Cierre'])
data_stock['Año'] = data_stock['Fecha'].dt.year
promedio_cierre_por_anio = data_stock.groupby('Año')['Cierre'].mean()
mediana_cierre_por_anio = data_stock.groupby('Año')['Cierre'].median()


ratio_ps_anual_promedio = promedio_cierre_por_anio / revenue_per_share
ratio_ps_anual_mediana = mediana_cierre_por_anio / revenue_per_share

ratio_ps_anual_promedio = ratio_ps_anual_promedio.dropna()
ratio_ps_anual_mediana = ratio_ps_anual_mediana.dropna()

# mediana_ratio_ps = ratio_ps_anual.median()



In [ ]:
# Asegurar orden e índice correcto
ratio_ps_anual_promedio = ratio_ps_anual_promedio.sort_index()
ratio_ps_anual_mediana  = ratio_ps_anual_mediana.sort_index()

ratio_ps_anual_promedio.index = ratio_ps_anual_promedio.index.astype(int)
ratio_ps_anual_mediana.index  = ratio_ps_anual_mediana.index.astype(int)

# Mediana móvil 10 años (incluye el año actual)
df_mediana_10y_promedio = (
    ratio_ps_anual_promedio
        .rolling(window=10, min_periods=10)
        .median()
        .dropna()
)

df_mediana_10y_mediana = (
    ratio_ps_anual_mediana
        .rolling(window=10, min_periods=10)
        .median()
        .dropna()
)

df_mediana_10y_mediana

2014    1.838871
2015    2.004972
2016    2.111615
2017    2.111615
2018    2.111615
2019    2.111615
2020    2.111615
2021    1.879373
2022    1.445925
2023    1.159355
dtype: float64

In [ ]:
revenue_per_share

2005     22.766351
2006     26.835449
2007     28.464459
2008     29.817993
2009     35.005699
2010     38.862194
2011     43.702119
2012     49.356803
2013     55.212088
2014     60.507995
2015     68.017554
2016     74.833732
2017     91.021191
2018    100.387715
2019    107.164234
2020     85.960937
2021    111.947486
2022    130.876601
2023    145.144436
dtype: float64

In [ ]:
Median_ps_value_promedio = df_mediana_10y_promedio * revenue_per_share

Median_ps_value_promedio_mediana = df_mediana_10y_mediana * revenue_per_share

Median_ps_value_promedio = Median_ps_value_promedio.dropna()
Median_ps_value_promedio_mediana = Median_ps_value_promedio_mediana.dropna()

## Graham Number

In [ ]:
df_tangible_book_value_per_share

,Año,Tangible_Book_Value_Per_Share
0,2019,69.18
1,2020,68.49
2,2021,77.06
3,2022,86.91
4,2023,98.05


In [ ]:
eps_without_nri = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'EPS without NRI'].iloc[0, 1:]

tangible_book_value_per_share = tangible_book_value_per_share.astype(float)
eps_without_nri = eps_without_nri.astype(float)

# Aplicar la fórmula: sqrt(22.5 * Tangible Book per Share * EPS without NRI)
graham_number = np.sqrt(22.5 * tangible_book_value_per_share * eps_without_nri)

# Mostrar el resultado
graham_number

NameError: name 'tangible_book_value_per_share' is not defined

## Peter Lynch Fair Value

El Valor Justo según Peter Lynch se aplica a empresas en crecimiento. El rango ideal para la tasa de crecimiento es entre 10% y 20% anual.

Peter Lynch considera que el valor justo del P/E (relación precio/utilidad) para una empresa en crecimiento es igual a su tasa de crecimiento, es decir, que el PEG = 1.

Las ganancias utilizadas en este cálculo son las del último año (TTM – trailing twelve months).

Para empresas que no son bancos, la tasa de crecimiento que se usa es el promedio de crecimiento del EBITDA por acción en los últimos 5 años.

Para bancos, se usa el promedio de crecimiento del Valor en Libros por acción en los últimos 5 años.

Si la tasa de crecimiento a 5 años es mayor al 25% anual, se limita a 25%.
Si la tasa de crecimiento a 5 años es menor al 5% anual, no se calcula el Valor Justo de Peter Lynch.

In [ ]:
peg_ratio = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'PEG Ratio'].iloc[0, 6:]
ebitda5anios = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'EBITDA'].iloc[0, 6:]
ebitda5anioscrecimiento = round(ebitda5anios.pct_change().mean(),3)*100
ultimo_valor_peg_ratio = peg_ratio.iloc[-1]
ultimo_eps_without_nri = eps_without_nri.iloc[-1]


peter_lynch_value_1 = 1 * ebitda5anioscrecimiento * ultimo_eps_without_nri
peter_lynch_value_2 = ultimo_valor_peg_ratio * ebitda5anioscrecimiento * ultimo_eps_without_nri

peter_lynch_value_min = min(peter_lynch_value_1, peter_lynch_value_2)
peter_lynch_value_min

130.0992

## DCF (FCF BASED)

In [ ]:
discount_rate = cetes + market_risk_premium

growth_rates = free_cash_flow.pct_change().dropna()
mediana_growth = growth_rates.median()
weighted_growth = (growth_rates * free_cash_flow.shift(1)).sum() / free_cash_flow.shift(1).sum()

In [ ]:
# Lista de valores (puedes reemplazar estos valores con los que ya tienes calculados)
valores = [free_cash_flow_crecimiento, cagr_free_cash_flow, mediana_growth, weighted_growth] 

# Filtrar los valores que están entre 5% y 25%
valores_filtrados = [v for v in valores if 0.05 <= v <= 0.25]

# Aplicar la lógica
if not valores_filtrados:  # Si no hay valores entre 5% y 25%
    resultado = 0.05  # El valor será 5%
else:
    resultado = min(valores_filtrados)  # Escoger el más bajo entre 5% y 25%

# Si todos los valores están por encima del 25%, el resultado será 25%
if all(v > 0.25 for v in valores):
    resultado = 0.25
# Mostrar el resultado
g1 = resultado


In [ ]:
y1 = 10
g2 = 0.04
y2 = 10
fcf_per_share = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Free Cash Flow per Share'].iloc[0, 1:]
fcf_per_share = fcf_per_share.iloc[-1]
d =  discount_rate
x = (1+g1)/(1+d)
y = (1+g2)/(1+d)

In [ ]:
intrinsic_value_fcf = fcf_per_share * (
    x * (1 - x**10) / (1 - x) +
    x**10 * y * (1 - y**10) / (1 - y)
)

intrinsic_value_fcf

186.62723453474717

## DCF(Earnigs Based)

In [ ]:
eps_without_nri_pct_change = round(eps_without_nri.pct_change().mean(),3)

inicio_eps_without_nri = eps_without_nri.iloc[0]
final_eps_without_nri = eps_without_nri.iloc[-1]
n_eps_without_nri = len(eps_without_nri)
cagr_eps_without_nri = round((final_eps_without_nri / inicio_eps_without_nri)**(1/n_eps_without_nri) - 1,3)

growth_rates_eps = eps_without_nri.pct_change().dropna()
mediana_growth_eps = growth_rates_eps.median()

weighted_growth_eps = (growth_rates_eps * eps_without_nri.shift(1)).sum() / eps_without_nri.shift(1).sum()


In [ ]:
# Lista de valores (puedes reemplazar estos valores con los que ya tienes calculados)
valores = [eps_without_nri_pct_change, cagr_eps_without_nri, mediana_growth_eps, weighted_growth_eps] 

# Filtrar los valores que están entre 5% y 25%
valores_filtrados = [v for v in valores if 0.05 <= v <= 0.25]

# Aplicar la lógica
if not valores_filtrados:  # Si no hay valores entre 5% y 25%
    resultado = 0.05  # El valor será 5%
else:
    resultado = min(valores_filtrados)  # Escoger el más bajo entre 5% y 25%

# Si todos los valores están por encima del 25%, el resultado será 25%
if all(v > 0.25 for v in valores):
    resultado = 0.25
# Mostrar el resultado
g1 = resultado

In [ ]:
y1 = 10
g2 = 0.04
y2 = 10
fcf_per_share = df_anual_filtrado[df_anual_filtrado['Fiscal Period'] == 'Free Cash Flow per Share'].iloc[0, 1:]
fcf_per_share = fcf_per_share.iloc[-1]
d =  discount_rate
x = (1+g1)/(1+d)
y = (1+g2)/(1+d)

In [ ]:

eps_without_nri_last = eps_without_nri.iloc[-1]

intrinscic_value_eb = eps_without_nri_last * (
    x * (1 - x**10) / (1 - x) +
    x**10 * y * (1 - y**10) / (1 - y)
)
intrinscic_value_eb

175.41062658965876

## Informacion trimestral calculos

## Dividend Discount Model

Solo funciona en acciones que siempre han ofrecido dividendo

In [ ]:
dividendos_ultimos_12_trimestres = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Dividends per Share'].iloc[0, 1:]
dividendos_ultimos_12_trimestres = dividendos_ultimos_12_trimestres.iloc[-12:]

# Calcular el crecimiento trimestre a trimestre
dividendos_filtrados = dividendos_ultimos_12_trimestres[dividendos_ultimos_12_trimestres != 0]

crecimiento_dividendo_trim = dividendos_filtrados.pct_change()

# Reemplazar NaN o valores no definidos con 0
crecimiento_dividendo_trim = crecimiento_dividendo_trim.fillna(0)

promedio_crecimiento_dividendo_trim = crecimiento_dividendo_trim.mean()

g_trim = round(promedio_crecimiento_dividendo_trim,4)

ultimo_dividendo_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Dividends per Share'].iloc[0, 1:]
ultimo_dividendo_trim = ultimo_dividendo_trim.iloc[-1]  # Último dividendo

In [ ]:
d1_trim = ultimo_dividendo_trim * (1 + g_trim)
p0_trim= d1_trim/(r-g_trim)
#Si salga negativo entonces no puede usarse XD
p0_trim

-10.66745632928975

## Book value per share

In [ ]:
total_assets_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Assets'].iloc[0, 1:]

total_assets_trim = total_assets_trim.iloc[-4:].mean()
total_liabilities_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]
total_liabilities_trim = total_liabilities_trim.iloc[-4:].mean()
total_shares_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Shares Outstanding (Basic Average)'].iloc[0, 1:]
total_shares_trim = total_shares_trim.iloc[-1]


In [ ]:
book_value_trim = total_assets_trim - total_liabilities_trim

# Calcular el valor en libros por acción (Book Value per Share) para cada año
book_value_per_share_trim = book_value_trim / total_shares_trim

book_value_per_share_trim

108.9438117037016

## Liquidation Value

In [ ]:
ratio_recuperacion_efectivo_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Cash, Cash Equivalents, Marketable Securities'].iloc[0, 1:] * 0.95

ratio_recuperacion_efectivo_trim = ratio_recuperacion_efectivo_trim.iloc[-4:].mean() 

ratio_recuperacion_cc_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Receivables'].iloc[0, 1:] * 0.75

ratio_recuperacion_cc_trim = ratio_recuperacion_cc_trim.iloc[-4:].mean()

ratio_recuperacion_inv_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Inventories'].iloc[0, 1:] * 0.60

ratio_recuperacion_inv_trim = ratio_recuperacion_inv_trim.iloc[-4:].mean()

ratio_recuperacion_ppe_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Property, Plant and Equipment'].iloc[0, 1:] * 0.70

ratio_recuperacion_ppe_trim = ratio_recuperacion_ppe_trim.iloc[-4:].mean()

ratio_recuperacion_act_tangibles_trim =df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Investments And Advances'].iloc[0, 1:] + df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Other Long Term Assets'].iloc[0, 1:] *.30

ratio_recuperacion_act_tangibles_trim = ratio_recuperacion_act_tangibles_trim.iloc[-4:].mean()

total_liabilities_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]

total_liabilities_trim = total_liabilities_trim.iloc[-4:].mean()

In [ ]:
liquidation_value_trim = ratio_recuperacion_efectivo_trim + ratio_recuperacion_cc_trim + ratio_recuperacion_inv_trim + ratio_recuperacion_ppe_trim + ratio_recuperacion_act_tangibles_trim - total_liabilities_trim

liquidation_value_per_share_trim = liquidation_value_trim / total_shares_trim
liquidation_value_per_share_trim

26.627272989573417

## EPV

In [ ]:
margen_op_20_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Operating Margin %'].iloc[0, 1:]/100

margen_op_20_trim = margen_op_20_trim.iloc[-20:]

ebit_promedio_trim=margen_op_20_trim.apply(pd.to_numeric, errors='coerce').mean()

ebit_promedio_redondeado_trim = round(ebit_promedio_trim, 4)

ebit_promedio_redondeado_trim

0.1385

In [ ]:
ventas_20_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]
ventas_20_trim  = ventas_20_trim.iloc[-20:]
ventas_20_trim_promedio = ventas_20_trim.apply(pd.to_numeric, errors='coerce').mean()
ventas_20_trim_promedio_redondeado = round(ventas_20_trim_promedio, 4)*4
ventas_20_trim_promedio_redondeado

167002.9864

In [ ]:
SGyA_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Selling, General, & Admin. Expense'].iloc[0, 1:]
# Seleccionar las últimas 5 columnas del DataFrame
ultimas20_columnas = SGyA_trim.index[-20:]
# Obtener los valores de las últimas 5 columnas y multiplicar por 0.75
SGyA_ultimos_20_trim = SGyA[ultimas20_columnas] * 0.25
SGyA_promedio_trim = round(SGyA_ultimos_20_trim.mean(),2)*4
SGyA_promedio_trim

14273.8

In [ ]:
EBIT_normalizado_trim = round((ventas_20_trim_promedio_redondeado * ebit_promedio_redondeado_trim) + SGyA_promedio_trim,2)
EBIT_normalizado_trim

37403.71

In [ ]:
tax_rate_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Tax Rate %'].iloc[0, 1:]
# Seleccionar las últimas 5 columnas del DataFrame
ultimas20_columnas = tax_rate_trim.index[-20:]
tax_rate_ultimos_20 = tax_rate_trim[ultimas20_columnas]
tax_rate_ultimos_20_positivos = tax_rate_ultimos_20[tax_rate_ultimos_20 >= 0]
# Calcular el promedio de los valores positivos 
promedio_tax_rate_trim = tax_rate_ultimos_20_positivos.mean()
# Redondear el promedio a dos decimales
promedio_tax_rate_redondeado_trim = round(promedio_tax_rate_trim, 4)/100
promedio_tax_rate_redondeado_trim

0.24710000000000001

In [ ]:
after_tax_normalized_ebit_trim = EBIT_normalizado_trim * (1 - promedio_tax_rate_redondeado_trim )
after_tax_normalized_ebit_trim

28161.253259

In [ ]:
DDA_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Depreciation, Depletion and Amortization'].iloc[0, 1:]
# Seleccionar las últimas 5 columnas del DataFrame
ultimas20_columnas = DDA.index[-20:]

DDA_ultimos_20 = DDA[ultimas20_columnas]
DDA_promedio_trim = round(DDA_ultimos_20.mean(),2)*4
Depreciacion_en_exceso_trim= DDA_promedio_trim * 0.5 * promedio_tax_rate_redondeado_trim
Depreciacion_en_exceso_trim

1761.4424660000002

In [ ]:
Normalized_earnings_trim = after_tax_normalized_ebit_trim + Depreciacion_en_exceso_trim
Normalized_earnings_trim

29922.695725

In [ ]:
revenue_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]
revenue_20_trim = revenue_trim.iloc[-20:]
capex_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Capital Expenditure'].iloc[0, 1:]
capex_20_trim = capex_trim.iloc[-20:].abs()
gppe_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Gross Property, Plant and Equipment'].iloc[0, 1:]
gppe_20_trim = gppe_trim.iloc[-20:]
revenue_cambio_trim = revenue_20_trim.diff().fillna(0)
accumulated_depreciation_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == '  Accumulated Depreciation'].iloc[0, 1:]
accumulated_depreciation_20_trim = accumulated_depreciation_trim.iloc[-20:].abs()
net_ppe_trim = gppe_20_trim - accumulated_depreciation_20_trim
net_ppe_trim

0
Sep2018    46352.924
Dec2018    47115.104
Mar2019    58653.465
Jun2019    59708.868
Sep2019    60926.322
Dec2019    62089.555
Mar2021    62466.337
Jun2021    62591.313
Sep2021    62734.568
Dec2021    64087.176
Mar2022    63795.478
Jun2022    64658.275
Sep2022    65737.774
Dec2022    67906.815
Mar2023    67208.518
Jun2023     68709.42
Sep2023    70145.849
Dec2023    71945.557
Mar2024    72399.087
Jun2024    73304.396
dtype: object

In [ ]:
# 1. Calcular growth_capex solo si hay aumento en revenue
growth_capex_trim = (net_ppe_trim / revenue_20_trim) * revenue_cambio_trim
growth_capex_trim = growth_capex_trim.where(revenue_cambio_trim > 0, 0)

# 2. Calcular mantenimiento preliminar
maintenance_capex_trim = capex_20_trim - growth_capex_trim

# 3. Si revenue_cambio fue negativo o el resultado fue negativo, se usa todo el CAPEX
maintenance_capex_trim = maintenance_capex_trim.where(
    (revenue_cambio_trim > 0) & (maintenance_capex_trim >= 0),
    capex_20_trim
)

# 4. Promedio
maintenance_capex_promedio_trim = maintenance_capex_trim.mean()*4

maintenance_capex_promedio_trim

6884.989199999999

In [ ]:
re_trim = r
precio_de_la_accion = 97.30
E_trim = total_shares_trim * precio_de_la_accion

Deuda_corto_plazo_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Short-Term Debt & Capital Lease Obligation'].iloc[0, 1:]
Deuda_corto_plazo_trim = Deuda_corto_plazo_trim.iloc[-4:].mean()
Deuda_largo_plazo_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Long-Term Debt & Capital Lease Obligation'].iloc[0, 1:]
Deuda_largo_plazo_trim  = Deuda_largo_plazo_trim.iloc[-4:].mean()

D_trim = Deuda_corto_plazo_trim + Deuda_largo_plazo_trim

market_risk_premium = 0.06 ##Se toma 6% porque historicamente es el rendimiento que ha existido pero tambien podemos calcular expected market return - risk free rate 10yrs


Cost_of_equity = cetes + beta * market_risk_premium
interest_expense_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == '  Interest Expense'].iloc[0, 1:].abs()
interest_expense_trim = interest_expense_trim.iloc[-4:].mean()
rd_trim = interest_expense_trim / D_trim

WACC_trim = E_trim / (E_trim + D_trim) * Cost_of_equity + D_trim / (E_trim + D_trim) * rd_trim * (1 - promedio_tax_rate_redondeado_trim)
weigth_equity_trim = E_trim / (E_trim + D_trim)
weigth_debt_trim = D_trim / (E_trim + D_trim)
WACC_trim


0.1097949666124025

In [ ]:
epv_business_operation_trim = (Normalized_earnings_trim - maintenance_capex_promedio_trim) / WACC_trim
cash_and_equivalents_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Cash, Cash Equivalents, Marketable Securities'].iloc[0, 1:]
cash_and_equivalents_trim = cash_and_equivalents_trim.iloc[-4:].mean() 
total_shares_ultimo = total_shares_last_value
cash_and_equivalents_trim

22820.2725

In [ ]:
EPV_business_operations_trim = (Normalized_earnings_trim - maintenance_capex_promedio_trim) / WACC_trim
# Ajuste final con efectivo y deuda
EPV_final_trim = (EPV_business_operations_trim + cash_and_equivalents_trim - D_trim) / total_shares_ultimo
EPV_final_trim

142.32478303416846

## Net Current Asset Value

In [ ]:
total_current_assets_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Current Assets'].iloc[0, 1:]
total_current_assets_trim = total_current_assets_trim.iloc[-4:].mean() 

total_liabilities_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Liabilities'].iloc[0, 1:]
total_liabilities_trim = total_liabilities_trim.iloc[-4:].mean() 
minority_interest_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Minority Interest'].iloc[0, 1:]
minority_interest_trim = minority_interest_trim.iloc[-4:].mean() 
prefered_stock_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Preferred Stock'].iloc[0, 1:]
prefered_stock_trim = prefered_stock_trim.iloc[-4:].mean() 

In [ ]:
net_current_asset_value_trim = total_current_assets_trim - total_liabilities_trim - minority_interest_trim - prefered_stock_trim
net_current_asset_value_per_share_trim = round(net_current_asset_value_trim / total_shares_ultimo,2)
net_current_asset_value_per_share_trim

-2.67

## Tangible book value

In [ ]:
total_equity_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Stockholders Equity'].iloc[0, 1:]
total_equity_trim = total_equity_trim.iloc[-4:].mean() 
prefered_stock_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Preferred Stock'].iloc[0, 1:]
prefered_stock_trim = prefered_stock_trim.iloc[-4:].mean() 
intagible_assets_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Intangible Assets'].iloc[0, 1:]
intagible_assets_trim = intagible_assets_trim.iloc[-4:].mean() 

tangible_book_value_trim = total_equity_trim - prefered_stock_trim - intagible_assets_trim
tangible_book_value_per_share_trim = tangible_book_value_trim / total_shares_ultimo

tangible_book_value_per_share_trim

97.17981092180278

## Projected Free Cash Flow

In [ ]:
ebitda_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EBITDA'].iloc[0, 1:]
free_cash_flow_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Free Cash Flow'].iloc[0, 1:]
ebitda_crecimiento_trim = round(ebitda_trim.pct_change().mean(),3)
free_cash_flow_crecimiento_trim = round(free_cash_flow_trim.pct_change().mean(),3)
revenue_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Revenue'].iloc[0, 1:]
revenue_crecimiento_trim = round(revenue_trim.pct_change().mean(),3)

##Ahora calculado con CAGR
inicio_ebitda_trim = ebitda_trim.iloc[0]
final_ebitda_trim = ebitda_trim.iloc[-1]
n_ebitda_trim = len(ebitda_trim)
cagr_ebitda_trim = round((final_ebitda_trim / inicio_ebitda_trim)**(1/n_ebitda_trim) - 1,3)

inicio_free_cash_flow_trim = free_cash_flow_trim.iloc[0]
final_free_cash_flow_trim = free_cash_flow_trim.iloc[-1]
n_free_cash_flow_trim = len(free_cash_flow_trim)
cagr_free_cash_flow_trim = (final_free_cash_flow_trim / inicio_free_cash_flow_trim)**(1/n_free_cash_flow_trim) - 1

inicio_revenue_trim = revenue_trim.iloc[0]
final_revenue_trim = revenue_trim.iloc[-1]
n_revenue_trim = len(revenue_trim)
cagr_revenue_trim = (final_revenue_trim / inicio_revenue_trim)**(1/n_revenue_trim) - 1

In [ ]:
##Obtener el valor de crecimiento mas bajo pero que este entre 4 y 11%
variables_trim = {
    'ebitda_crecimiento': ebitda_crecimiento_trim,
    'free_cash_flow_crecimiento': free_cash_flow_crecimiento_trim,
    'revenue_crecimiento': revenue_crecimiento_trim,
    'cagr_ebitda': cagr_ebitda_trim,
    'cagr_free_cash_flow': cagr_free_cash_flow_trim,
    'cagr_revenue': cagr_revenue_trim
}

# Filtrar solo los valores numéricos reales y dentro del rango
variables_filtradas_trim = {
    k: v for k, v in variables_trim.items()
    if isinstance(v, (int, float)) and 0.04 <= v <= 0.11
}

# Determinar el valor mínimo o usar 5% por defecto
if variables_filtradas_trim:
    variable_minima_trim = min(variables_filtradas_trim, key=variables_filtradas_trim.get)
    valor_minimo_trim = variables_filtradas_trim[variable_minima_trim]
else:
    variable_minima_trim = 'default_5%'
    valor_minimo_trim = 0.05

In [ ]:
growth_assumption_trim = valor_minimo_trim * 100
growth_multiple_trim = 8.3459 * (1.07 ** (growth_assumption_trim - 4))
growth_multiple_trim

8.750680372830846

In [ ]:
free_cash_flow_6anios_trim= free_cash_flow_trim.iloc[-24:]
free_cash_promedio_trim = free_cash_flow_6anios_trim.mean()*4
free_cash_flow_4trimestres_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Free Cash Flow'].iloc[0, 1:]
free_cash_flow_4trimestres_trim = free_cash_flow_4trimestres_trim.iloc[-4:].mean()

total_stockholders_equity_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Total Stockholders Equity'].iloc[0, 1:]
total_stockholders_equity_trim = total_stockholders_equity_trim.iloc[-4:].mean() 

fcf_ajustado_trim = (6*free_cash_promedio_trim)+(0.75*free_cash_flow_4trimestres_trim)
fcf_ajustado_trim = fcf_ajustado_trim / 6.75

factor_inflacion_trim = (1 + data_inflacion_mexico_promedio) ** 3

fcf_ajustado_inflacion_trim = fcf_ajustado_trim * factor_inflacion_trim


In [ ]:
intrinscic_value_trim = (growth_multiple_trim * fcf_ajustado_inflacion_trim+ total_stockholders_equity_trim *0.7) / total_shares_ultimo
intrinscic_value_trim

158.28337920429638

## Median PS Value

### No se puede hacer con valores trimestrales, debido a que se ocupan media de los ultimos 10 años

## Graham Number

In [ ]:
eps_without_nri_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EPS without NRI'].iloc[0, -4:].mean()*4

graham_number_trim = np.sqrt(22.5 * tangible_book_value_per_share_trim * eps_without_nri_trim)

graham_number_trim

186.70273773728132

## Peter Lynch Fair Value

In [ ]:
ultimo_peg_ratio_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'PEG Ratio'].iloc[0,-1:]

# Tomar últimos 20 trimestres de EBITDA
ebitda5anios_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EBITDA'].iloc[0, -20:]

# Quitar trimestres de 2020
cols_validas = [col for col in ebitda5anios_trim.index if not col.endswith('2020')]
ebitda5anios_trim = ebitda5anios_trim[cols_validas]

# Calcular crecimiento trimestral promedio
crecimiento_trim = ebitda5anios_trim.pct_change().mean()

crecimiento_trim_limited = min(crecimiento_trim, 0.25) *100

peg_ratio_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'PEG Ratio'].iloc[0, 6:]
ultimo_valor_peg_ratio_trim = peg_ratio_trim.iloc[-1]

peter_lynch_value_1_trim = 1 * crecimiento_trim_limited * eps_without_nri_trim
peter_lynch_value_2_trim = ultimo_valor_peg_ratio_trim * crecimiento_trim_limited * eps_without_nri_trim

peter_lynch_value_min_trim = min(peter_lynch_value_1_trim, peter_lynch_value_2_trim)

peter_lynch_value_min_trim


183.333

## DCF (FCF BASED)

In [ ]:
discount_rate

0.14350000000000002

In [ ]:
growth_rates_trim = free_cash_flow_trim.pct_change().dropna()
mediana_growth_trim = growth_rates_trim.median()
weighted_growth_trim = (growth_rates_trim * free_cash_flow_trim.shift(1)).sum() / free_cash_flow_trim.shift(1).sum()
weighted_growth_trim

0.06529767330517841

In [ ]:
# Lista de valores (puedes reemplazar estos valores con los que ya tienes calculados)
valores_trim = [free_cash_flow_crecimiento_trim, cagr_free_cash_flow_trim, mediana_growth_trim, weighted_growth_trim] 

valores_trim = [float(v.real) if isinstance(v, complex) else float(v) for v in valores_trim]

# Filtrar los valores que están entre 5% y 25%
valores_filtrados_trim = [v for v in valores_trim if 0.05 <= v <= 0.25]

# Aplicar la lógica
if not valores_filtrados_trim:  # Si no hay valores entre 5% y 25%
    resultado_trim = 0.05  # El valor será 5%
else:
    resultado_trim = min(valores_filtrados_trim)  # Escoger el más bajo entre 5% y 25%

# Si todos los valores están por encima del 25%, el resultado será 25%
if all(v > 0.25 for v in valores_trim):
    resultado_trim = 0.25
# Mostrar el resultado
g1_trim = resultado_trim
g1_trim

0.06529767330517841

In [ ]:
fcf_per_share_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'Free Cash Flow per Share'].iloc[0, -4:].mean()*4

d_trim =  discount_rate
x_trim = (1+g1_trim)/(1+d_trim)
y_trim = (1+g2)/(1+d_trim)

In [ ]:
intrinsic_value_trim_fcf = fcf_per_share_trim * (
    x_trim * (1 - x_trim**10) / (1 - x_trim) +
    x_trim**10 * y_trim * (1 - y_trim**10) / (1 - y_trim)
)

intrinsic_value_trim_fcf

122.60782608832021

## DCF (Earnings Based)

In [ ]:
eps_without_nri_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EPS without NRI'].iloc[0, 1:]
eps_without_nri_trim = eps_without_nri_trim[~eps_without_nri_trim.index.str.contains('2020')]

eps_without_nri_pct_change_trim = round(eps_without_nri_trim.pct_change().mean(),3)

inicio_eps_without_nri_trim = eps_without_nri_trim.iloc[0]
final_eps_without_nri_trim = eps_without_nri_trim.iloc[-1]
n_eps_without_nri_trim = len(eps_without_nri_trim)
cagr_eps_without_nri_trim = round((final_eps_without_nri_trim / inicio_eps_without_nri_trim)**(1/n_eps_without_nri_trim) - 1,3)

growth_rates_eps_trim = eps_without_nri_trim.pct_change().dropna()
mediana_growth_eps_trim = growth_rates_eps_trim.median()

weighted_growth_eps_trim = (growth_rates_eps_trim * eps_without_nri_trim.shift(1)).sum() / eps_without_nri_trim.shift(1).sum()

In [ ]:
# Lista de valores (puedes reemplazar estos valores con los que ya tienes calculados)
valores_trim = [eps_without_nri_pct_change_trim, cagr_eps_without_nri_trim, mediana_growth_eps_trim, weighted_growth_eps_trim] 

# Filtrar los valores que están entre 5% y 25%
valores_filtrados_trim = [v for v in valores_trim if 0.05 <= v <= 0.25]

# Aplicar la lógica
if not valores_filtrados_trim:  # Si no hay valores entre 5% y 25%
    resultado_trim = 0.05  # El valor será 5%
else:
    resultado_trim = min(valores_filtrados_trim)  # Escoger el más bajo entre 5% y 25%

# Si todos los valores están por encima del 25%, el resultado será 25%
if all(v > 0.25 for v in valores_trim):
    resultado_trim = 0.25
# Mostrar el resultado
g1_trim = resultado_trim
g1_trim

0.05

In [ ]:
y1_trim = 10
g2_trim = 0.04
y2_trim = 10
eps_without_nri_last_trim = df_trimestral_filtrado[df_trimestral_filtrado['Fiscal Period'] == 'EPS without NRI'].iloc[0, -4:].mean()*4

d_trim =  discount_rate
x_trim = (1+g1_trim)/(1+d_trim)
y_trim = (1+g2_trim)/(1+d_trim)


In [ ]:
intrinscic_value_eb_trim = eps_without_nri_last_trim * (
    x_trim * (1 - x_trim**10) / (1 - x_trim) +
    x_trim**10 * y_trim * (1 - y_trim**10) / (1 - y_trim)
)
intrinscic_value_eb_trim

144.56818014544154

## Tabla con valores de acciones

In [ ]:
p0
book_value_per_share_anual  ## ya viene todos los años
liquidation_value_per_share ## ya viene todos los años
EPV_final
net_current_asset_value_per_share
tangible_book_value_per_share ## ya viene todos los años
intrinscic_value
Median_ps_value
graham_number  ## ya viene todos los años
peter_lynch_value_1
peter_lynch_value_2
intrinsic_value_fcf
intrinscic_value_eb

###Valore de los trimestres
p0_trim
book_value_per_share_trim
liquidation_value_per_share_trim
EPV_final_trim
net_current_asset_value_per_share_trim
tangible_book_value_per_share_trim
intrinscic_value_trim
graham_number_trim
peter_lynch_value_1_trim
peter_lynch_value_2_trim
intrinsic_value_trim_fcf
intrinscic_value_eb_trim

144.56818014544154